Загрузим обученные модели. Будем испльзовать две лучшие модели ML, и три лучшие модели на нейронных сетях.
Объединим их в ансамбли.
Проверим, как они торгуют на бэктесте.

# Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import talib
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import yfinance as yf

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from IPython.display import clear_output

%matplotlib inline

pd.set_option('display.max_columns', None)

In [2]:
# Загрузим из файла
eth_df = pd.read_csv('eth_data_2025_01_01_to_2025_02_19.csv', index_col=0, parse_dates=True)
eth_df 

,Close,High,Low,Open,Volume,btc_close
Datetime,,,,,,
2025-01-01 00:00:00+00:00,3337.285889,3337.397217,3330.587402,3331.279785,0,93330.968750
2025-01-01 00:15:00+00:00,3340.867188,3344.197266,3340.129639,3340.129639,203587584,93534.460938
2025-01-01 00:30:00+00:00,3352.541260,3353.072021,3341.765869,3341.765869,226148352,93711.296875
2025-01-01 00:45:00+00:00,3358.024658,3358.024658,3347.942627,3349.490234,0,94256.054688
2025-01-01 01:00:00+00:00,3354.194336,3357.458740,3349.236572,3357.458740,0,93956.867188
...,...,...,...,...,...,...
2025-02-19 22:45:00+00:00,2722.684570,2728.985596,2711.882812,2711.882812,762358784,96537.156250
2025-02-19 23:00:00+00:00,2721.705078,2729.082275,2721.705078,2723.190430,709018624,96615.671875
2025-02-19 23:15:00+00:00,2722.963623,2725.658203,2721.704834,2721.704834,1750604800,96667.070312


# Подготовка данных

In [3]:
def fill_missing_values(data):
    """
    Функция заполняет пропущенные значения в DataFrame предыдущими значениями 
    и удаляет дубликаты по индексу.
    """
    df = data.copy()

    # Проверка и удаление дубликатов по индексу
    if df.index.duplicated().any():
        print("Обнаружены дубликаты по индексу. Они будут удалены.")
        df = df[~df.index.duplicated(keep='first')]
    else:
        print("Дубликатов не обнаружено.")

    # Проверка на наличие пропущенных значений и нулевых значений
    if df.isnull().any().any() or (df == 0).any().any():
        print("Обнаружены пропущенные или нулевые значения. Выполняется заполнение предыдущими значениями...")

        # Заменяем нули на NaN, чтобы их можно было заполнить
        df.replace(0, np.nan, inplace=True)
        
        # Заполняем пропущенные значения предыдущими
        df = df.fillna(method='ffill')
        
        # Если в начале есть NaN, заполняем их последующими значениями
        df = df.fillna(method='bfill')
    else:
        print("Пропущенные значения отсутствуют.")

    return df

In [4]:
# Функция для добавления лагов

def add_lags(data, columns, lags):
    """Функция для добавления лагов

    Args:
        data (pd.Dataframe): pandas Dataframe с данными по активу
        columns (List): список колонорк для расчета лагов
        lags (List): список шагов назад для расчета лагов
    """
    df = data.copy()

    for lag in lags: # Добавляем лаги по шагам назад из lags
        for column in columns:
            new_col_name = f'{column}_lag_{lag}'
            df[new_col_name] = df[column].shift(lag)
    
    return df

In [5]:
# Функция для добавления индикаторов

def add_indicators(data, columns, windows_SMA, windows_RSI):
    """Функция для добавления индикаторов SMA и RSI

    Args:
        data (pd.Dataframe): pandas Dataframe с данными по активу
        columns (List): список колонорк для расчета индикаторов
        windows_SMA (List): список окон для расчета MA
        windows_RSI (List): список окон для расчета RSI
    """
    df = data.copy()

    for window in windows_SMA: # Расчитываем и добавляем SMA по указанным колонкам
        for column in columns:
            new_col_name = f'{column}_SMA_{window}'
            df[new_col_name] = df[column].rolling(window).mean()

    for window in windows_RSI: # Расчитываем и добавляем RSI по указанным колонкам
        for column in columns:
            new_col_name = f'{column}_RSI_{window}'
            df[new_col_name] = talib.RSI(df[column], timeperiod=window)
    
    return df

In [6]:
# Функция для добавления статистик

def add_stats_features(data, columns, windows):
    """Функция для добавления статистик, min, max, std за периоды windows

    Args:
        data (pd.Dataframe): pandas Dataframe с данными по активу
        columns (List): список колонорк для расчета статистик
        windows (List): список окон для расчета статистик
    """   
    df = data.copy()

    for column in columns:
        for window in windows:
            # Скользящее среднее
            df[f'{column}_mean_{window}'] = df[column].rolling(window=window).mean()
            
            # Скользящая медиана
            df[f'{column}_median_{window}'] = df[column].rolling(window=window).median()
            
            # Скользящий минимум
            df[f'{column}_min_{window}'] = df[column].rolling(window=window).min()
            
            # Скользящий максимум
            df[f'{column}_max_{window}'] = df[column].rolling(window=window).max()
            
            # Скользящее стандартное отклонение
            df[f'{column}_std_{window}'] = df[column].rolling(window=window).std()
            
            # Скользящий размах (макс - мин)
            df[f'{column}_range_{window}'] = df[f'{column}_max_{window}'] - df[f'{column}_min_{window}']

    return df

In [7]:
# Функция для добавления разницы между higt и low

def add_hight_low_dif(data):
    df = data.copy()
    df['High-Low'] = df['High'] - df['Low']
    return df

In [8]:
# Функция для добавления трендовых фичей

def add_trend_feat(data, columns, windows):
    """Функция для трендовых фичей

    Args:
        data (pd.Dataframe): pandas Dataframe с данными по активу
        columns (List): список колонорк трендовых фичей
        windows (List): список окон трендовых фичей
    """   
    df = data.copy()
    for column in columns:
        for window in windows:
            # Отношение текущего значения к предыдущему (лаг = 1)
            df[f'{column}_ratio_1'] = df[column] / df[column].shift(1)
            
            # Логарифмическое изменение (логарифм отношения текущего значения к предыдущему)
            df[f'{column}_log_diff_1'] = np.log(df[column] / df[column].shift(1))
            
            # Momentum (разница между текущим значением и значением N периодов назад)
            df[f'{column}_momentum_{window}'] = df[column] - df[column].shift(window)
            
            # Rate of Change (ROC): процентное изменение за N периодов
            df[f'{column}_roc_{window}'] = (df[column] - df[column].shift(window)) / df[column].shift(window) * 100
    
    return df    

In [9]:
import pickle

# Функция для нормализации данных
def normalize_dataframe(data, scaler="StandardScaler", save_scaler=True, scaler_file="scaler.pkl"):
    """Функция нормализует DataFrame с использованием заданного скейлера.

    Args:
    df (pd.DataFrame): Датафрейм для нормализации.
    scaler (str or sklearn Scaler): Имя скейлера ("StandardScaler", "MinMaxScaler", "RobustScaler") 
                                     или уже обученный объект скейлера.
    save_scaler (bool): сохранять ли scaler в файл (по умолчанию: сохранять)
    scaler_file (str): Имя файла, куда сохраняется обученный скейлер (по умолчанию: "scaler.pkl").

    """
    df = data.copy()

    if isinstance(scaler, str):
        # Создаем скейлер на основе переданного имени
        if scaler == "StandardScaler":
            scaler = StandardScaler()
        elif scaler == "MinMaxScaler":
            scaler = MinMaxScaler()
        elif scaler == "RobustScaler":
            scaler = RobustScaler()
        else:
            raise ValueError(f"Неизвестный скейлер: {scaler}")

        # Обучаем скейлер на датафрейме
        scaler.fit(df)

        # Сохраняем обученный скейлер в файл
        if save_scaler:
            with open(scaler_file, "wb") as f:
                pickle.dump(scaler, f)
    elif hasattr(scaler, "transform"):
        # Если передан обученный скейлер, просто используем его
        pass
    else:
        raise ValueError("Передан некорректный скейлер. Должна быть строка 'StandardScaler', 'MinMaxScaler', 'RobustScaler' или объект sklearn Scaler.")

    # Применяем скейлер к датафрейму
    normalized_data = scaler.transform(df)

    # Возвращаем нормализованный датафрейм и обученный скейлер
    return pd.DataFrame(normalized_data, columns=df.columns, index=df.index), scaler

In [10]:
# Функция для загрузки скейлера из файла
def load_scaler(scaler_file):
    """Загружает скейлер из указанного файла.

    Args:
    scaler_file (str): Путь к файлу, содержащему сохранённый скейлер.

    Return:
    sklearn Scaler: Загруженный объект скейлера.
    """
    with open(scaler_file, "rb") as f:
        scaler = pickle.load(f)
    return scaler

In [11]:
# Функция для добавления номеров месяцев, дней, часов

def add_datetime_features(data, add_month=True, add_weekday=True, add_hour=True):
    """Функция добавляет в DataFrame столбцы с номером месяца, дня недели и часа из индекса Datetime.
    
    Args:
    df (pd.DataFrame): DataFrame с индексом типа Datetime.
    add_month (bool): Добавлять ли столбец с номером месяца (1-12).
    add_weekday (bool): Добавлять ли столбец с номером дня недели (0 - понедельник, 6 - воскресенье).
    add_hour (bool): Добавлять ли столбец с номером часа (0-23).
    """
    df = data.copy()
    
    # Добавляем столбцы по выбору
    if add_month:
        df['Month'] = df.index.month
    if add_weekday:
        df['Weekday'] = df.index.weekday
    if add_hour:
        df['Hour'] = df.index.hour
    
    return df

In [12]:
def prepare_date(data, 
                 lags, 
                 windows_SMA, 
                 windows_RSI, 
                 windows_stats, 
                 windows_trend, 
                 add_month=True, 
                 add_weekday=True, 
                 add_hour=True):
    """
    Функция для добавления фичей и нормализации

    Args:
        data (pd.DataFrame): DataFrame с индексом типа Datetime.
        windows_SMA (List): список окон для индикатора SMA.
        windows_RSI (List): список окон для индикатора RSI.
        windows_stats (List): список окон для статистических фичей.
        windows_trend (List): список окон для трендовых фичей.
        add_month (bool): Добавлять ли столбец с номером месяца (1-12).
        add_weekday (bool): Добавлять ли столбец с номером дня недели (0 - понедельник, 6 - воскресенье).
        add_hour (bool): Добавлять ли столбец с номером часа (0-23).
    """

    df = data.copy()
    columns = df.columns

    df = fill_missing_values(df)
    df = add_lags(df, columns=columns, lags=lags)
    df = add_indicators(df, columns=['Close'], windows_SMA=windows_SMA, windows_RSI=windows_RSI)
    df = add_stats_features(df, columns=['Close', 'btc_close'], windows=windows_stats)
    df = add_hight_low_dif(df)
    df = add_trend_feat(df, columns=['Close', 'btc_close'], windows=windows_trend)
    df = add_datetime_features(df, add_month=add_month, add_weekday=add_weekday, add_hour=add_hour)
    df = df.dropna()

    return df

In [13]:
window_lag = [3, 9, 15, 30]
window_SMA = [6, 9, 30, 50]
window_RSI = [4, 9, 20]
windows_stats = [3, 10, 30, 50]
windows_trend = [3, 10, 30, 50]

In [14]:
eth_df_features = prepare_date(eth_df, lags=window_lag, windows_SMA=window_SMA, 
                               windows_RSI=window_RSI, windows_stats=windows_stats, 
                               windows_trend=windows_trend)

Дубликатов не обнаружено.
Обнаружены пропущенные или нулевые значения. Выполняется заполнение предыдущими значениями...


C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\176028078.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\176028078.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')


In [15]:
eth_df_features

,Close,High,Low,Open,Volume,btc_close,Close_lag_3,High_lag_3,Low_lag_3,Open_lag_3,Volume_lag_3,btc_close_lag_3,Close_lag_9,High_lag_9,Low_lag_9,Open_lag_9,Volume_lag_9,btc_close_lag_9,Close_lag_15,High_lag_15,Low_lag_15,Open_lag_15,Volume_lag_15,btc_close_lag_15,Close_lag_30,High_lag_30,Low_lag_30,Open_lag_30,Volume_lag_30,btc_close_lag_30,Close_SMA_6,Close_SMA_9,Close_SMA_30,Close_SMA_50,Close_RSI_4,Close_RSI_9,Close_RSI_20,Close_mean_3,Close_median_3,Close_min_3,Close_max_3,Close_std_3,Close_range_3,Close_mean_10,Close_median_10,Close_min_10,Close_max_10,Close_std_10,Close_range_10,Close_mean_30,Close_median_30,Close_min_30,Close_max_30,Close_std_30,Close_range_30,Close_mean_50,Close_median_50,Close_min_50,Close_max_50,Close_std_50,Close_range_50,btc_close_mean_3,btc_close_median_3,btc_close_min_3,btc_close_max_3,btc_close_std_3,btc_close_range_3,btc_close_mean_10,btc_close_median_10,btc_close_min_10,btc_close_max_10,btc_close_std_10,btc_close_range_10,btc_close_mean_30,btc_close_median_30,btc_close_min_30,btc_close_max_30,btc_close_std_30,btc_close_range_30,btc_close_mean_50,btc_close_median_50,btc_close_min_50,btc_close_max_50,btc_close_std_50,btc_close_range_50,High-Low,Close_ratio_1,Close_log_diff_1,Close_momentum_3,Close_roc_3,Close_momentum_10,Close_roc_10,Close_momentum_30,Close_roc_30,Close_momentum_50,Close_roc_50,btc_close_ratio_1,btc_close_log_diff_1,btc_close_momentum_3,btc_close_roc_3,btc_close_momentum_10,btc_close_roc_10,btc_close_momentum_30,btc_close_roc_30,btc_close_momentum_50,btc_close_roc_50,Month,Weekday,Hour
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-01-01 12:30:00+00:00,3343.072754,3344.783691,3341.038330,3343.430420,4.343808e+07,93679.632812,3334.746826,3339.170166,3334.012695,3337.341064,4.343808e+07,93267.125000,3325.237549,3328.333740,3323.114502,3323.114502,9.007104e+06,93254.468750,3330.404053,3333.293213,3328.243164,3331.517090,5.076992e+06,93259.992188,3339.198975,3339.198975,3334.000000,3334.000000,1.054720e+06,93447.375000,3336.257935,3331.594265,3332.004256,3337.854551,87.426735,68.418219,57.020899,3340.554118,3343.072754,3335.316406,3343.273193,4.537098,7.956787,3330.958594,3330.569214,3319.422119,3343.273193,8.375223,23.851074,3332.004256,3334.248535,3312.978760,3343.273193,7.791334,30.294434,3337.854551,3338.296997,3312.978760,3358.024658,10.136359,45.045898,93513.197917,93616.367188,93243.593750,93679.632812,235.617147,436.039062,93304.431250,93258.507812,93111.765625,93679.632812,195.346178,567.867188,93349.735156,93330.218750,92867.187500,93679.632812,214.242752,812.445312,93471.558125,93515.777344,92867.187500,94256.054688,255.901936,1388.867188,3.745361,0.999940,-0.000060,8.325928,0.249672,17.716309,0.532764,3.873779,0.116009,5.786865,0.173400,1.000676,0.000676,412.507812,0.442286,442.554688,0.474655,232.257812,0.248544,348.664062,0.373578,1,2,12
2025-01-01 12:45:00+00:00,3340.229736,3340.972900,3338.161133,3340.972900,4.343808e+07,93618.367188,3335.316406,3335.985107,3333.931885,3334.479492,4.343808e+07,93243.593750,3324.588379,3329.039795,3323.121826,3325.411865,9.007104e+06,93138.328125,3325.482666,3333.349854,3325.310059,3330.390137,1.912238e+08,93106.484375,3342.497070,3342.497070,3338.213135,3338.907959,9.932800e+06,93582.289062,3338.470459,3333.332194,3331.928678,3337.841802,67.214291,62.324199,54.907598,3342.191895,3343.072754,3340.229736,3343.273193,1.702232,3.043457,3332.457812,3334.465332,3319.422119,3343.273193,8.576754,23.851074,3331.928678,3334.248535,3312.978760,3343.273193,7.696461,30.294434,3337.841802,3338.296997,3312.978760,3358.024658,10.132893,45.045898,93638.122396,93618.367188,93616.367188,93679.632812,35.962981,63.265625,93340.821094,93264.835938,93111.765625,93679.632812,217.628245,567.867188,93350.937760,93330.218750,92867.187500,93679.632812,215.689528,812.445312,93473.236250,93515.777344,92867.187500,94256.054688,256.597023,1388.867188,2.811768,0.999150,-0.000851,4.913330,0.14

In [16]:
def add_target_column(df, close_col='Close', threshold=1):
    """
    Добавляет колонку target в датафрейм на основе процентного изменения цены через час.

    - df: Исходный датафрейм.
    - close_col: Название колонки с ценами закрытия.
    - threshold: Пороговое значение процента для классификации.
    """
    # Рассчитываем процентное изменение цены через час
    df['price_change_pct'] = df[close_col].pct_change(periods=4).shift(-4) * 100

    # Создаем колонку target на основе условий
    df['target'] = np.select(
        [
            df['price_change_pct'] < -threshold,  # Цена упала на threshold и больше
            (df['price_change_pct'] >= -threshold) & (df['price_change_pct'] <= threshold),  # Цена в пределах threshold
            df['price_change_pct'] > threshold  # Цена выросла на threshold и больше
        ],
        [0, 1, 2],  # Соответствующие классы
        default=np.nan  # Если значение NaN (последние 4 строки)
    )

    # Удаляем временную колонку price_change_pct
    df.drop(columns=['price_change_pct'], inplace=True)

    # Удаляем строки с NaN в target (последние 4 строки)
    df.dropna(subset=['target'], inplace=True)

    # Преобразуем target в целые числа
    df['target'] = df['target'].astype(int)

    return df

In [17]:
eth_df_features = add_target_column(eth_df_features, threshold=0.2)

eth_df_features

,Close,High,Low,Open,Volume,btc_close,Close_lag_3,High_lag_3,Low_lag_3,Open_lag_3,Volume_lag_3,btc_close_lag_3,Close_lag_9,High_lag_9,Low_lag_9,Open_lag_9,Volume_lag_9,btc_close_lag_9,Close_lag_15,High_lag_15,Low_lag_15,Open_lag_15,Volume_lag_15,btc_close_lag_15,Close_lag_30,High_lag_30,Low_lag_30,Open_lag_30,Volume_lag_30,btc_close_lag_30,Close_SMA_6,Close_SMA_9,Close_SMA_30,Close_SMA_50,Close_RSI_4,Close_RSI_9,Close_RSI_20,Close_mean_3,Close_median_3,Close_min_3,Close_max_3,Close_std_3,Close_range_3,Close_mean_10,Close_median_10,Close_min_10,Close_max_10,Close_std_10,Close_range_10,Close_mean_30,Close_median_30,Close_min_30,Close_max_30,Close_std_30,Close_range_30,Close_mean_50,Close_median_50,Close_min_50,Close_max_50,Close_std_50,Close_range_50,btc_close_mean_3,btc_close_median_3,btc_close_min_3,btc_close_max_3,btc_close_std_3,btc_close_range_3,btc_close_mean_10,btc_close_median_10,btc_close_min_10,btc_close_max_10,btc_close_std_10,btc_close_range_10,btc_close_mean_30,btc_close_median_30,btc_close_min_30,btc_close_max_30,btc_close_std_30,btc_close_range_30,btc_close_mean_50,btc_close_median_50,btc_close_min_50,btc_close_max_50,btc_close_std_50,btc_close_range_50,High-Low,Close_ratio_1,Close_log_diff_1,Close_momentum_3,Close_roc_3,Close_momentum_10,Close_roc_10,Close_momentum_30,Close_roc_30,Close_momentum_50,Close_roc_50,btc_close_ratio_1,btc_close_log_diff_1,btc_close_momentum_3,btc_close_roc_3,btc_close_momentum_10,btc_close_roc_10,btc_close_momentum_30,btc_close_roc_30,btc_close_momentum_50,btc_close_roc_50,Month,Weekday,Hour,target
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-01-01 12:30:00+00:00,3343.072754,3344.783691,3341.038330,3343.430420,4.343808e+07,93679.632812,3334.746826,3339.170166,3334.012695,3337.341064,4.343808e+07,93267.125000,3325.237549,3328.333740,3323.114502,3323.114502,9.007104e+06,93254.468750,3330.404053,3333.293213,3328.243164,3331.517090,5076992.0,93259.992188,3339.198975,3339.198975,3334.000000,3334.000000,1.054720e+06,93447.375000,3336.257935,3331.594265,3332.004256,3337.854551,87.426735,68.418219,57.020899,3340.554118,3343.072754,3335.316406,3343.273193,4.537098,7.956787,3330.958594,3330.569214,3319.422119,3343.273193,8.375223,23.851074,3332.004256,3334.248535,3312.978760,3343.273193,7.791334,30.294434,3337.854551,3338.296997,3312.978760,3358.024658,10.136359,45.045898,93513.197917,93616.367188,93243.593750,93679.632812,235.617147,436.039062,93304.431250,93258.507812,93111.765625,93679.632812,195.346178,567.867188,93349.735156,93330.218750,92867.187500,93679.632812,214.242752,812.445312,93471.558125,93515.777344,92867.187500,94256.054688,255.901936,1388.867188,3.745361,0.999940,-0.000060,8.325928,0.249672,17.716309,0.532764,3.873779,0.116009,5.786865,0.173400,1.000676,0.000676,412.507812,0.442286,442.554688,0.474655,232.257812,0.248544,348.664062,0.373578,1,2,12,0
2025-01-01 12:45:00+00:00,3340.229736,3340.972900,3338.161133,3340.972900,4.343808e+07,93618.367188,3335.316406,3335.985107,3333.931885,3334.479492,4.343808e+07,93243.593750,3324.588379,3329.039795,3323.121826,3325.411865,9.007104e+06,93138.328125,3325.482666,3333.349854,3325.310059,3330.390137,191223808.0,93106.484375,3342.497070,3342.497070,3338.213135,3338.907959,9.932800e+06,93582.289062,3338.470459,3333.332194,3331.928678,3337.841802,67.214291,62.324199,54.907598,3342.191895,3343.072754,3340.229736,3343.273193,1.702232,3.043457,3332.457812,3334.465332,3319.422119,3343.273193,8.576754,23.851074,3331.928678,3334.248535,3312.978760,3343.273193,7.696461,30.294434,3337.841802,3338.296997,3312.978760,3358.024658,10.132893,45.045898,93638.122396,93618.367188,93616.367188,93679.632812,35.962981,63.265625,93340.821094,93264.835938,93111.765625,93679.632812,217.628245,567.867188,93350.937760,93330.218750,92867.187500,93679.632812,215.689528,812.445312,93473.236250,93515.777344,92867.187500,94256.054688,256.597023,1388.867188,2.811768,0.999150,-0.000851,4.91333

# Загрузка моделей ML

In [18]:
# Загрузка модели
with open('best_ML/best_model_xgboost.pkl', 'rb') as file:
    best_model_xgboost = pickle.load(file)

# Загрузка модели
with open('best_ML/best_model_LGBMC.pkl', 'rb') as file:
    best_model_LGBMC = pickle.load(file)

# Загрузка моделей NN

In [19]:
class TSmixer(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(TSmixer, self).__init__()
        
        # Входной слой
        self.input_proj = nn.Linear(input_size, hidden_size)
        
        # Сверточные слои
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=hidden_size, out_channels=hidden_size, kernel_size=3, padding=1)
            for _ in range(num_layers)
        ])
        
        # Слои внимания
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=hidden_size, num_heads=4)
            for _ in range(num_layers)
        ])
        
        # Выходной слой
        self.output_proj = nn.Linear(hidden_size, num_classes)
        
        # Нормализация
        self.layer_norm = nn.LayerNorm(hidden_size)
        
        # Dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # x shape: (batch_size, window_size, input_size)
        
        # Проекция входных данных
        x = self.input_proj(x)  # (batch_size, window_size, hidden_size)
        x = x.permute(1, 0, 2)  # (window_size, batch_size, hidden_size)
        
        # Применяем сверточные слои и механизмы внимания
        for conv, attention in zip(self.conv_layers, self.attention_layers):
            # Сверточный слой
            residual = x
            x = x.permute(1, 2, 0)  # (batch_size, hidden_size, window_size)
            x = conv(x)  # (batch_size, hidden_size, window_size)
            x = x.permute(2, 0, 1)  # (window_size, batch_size, hidden_size)
            x = self.layer_norm(x + residual)
            
            # Механизм внимания
            residual = x
            x, _ = attention(x, x, x)  # (window_size, batch_size, hidden_size)
            x = self.layer_norm(x + residual)
            x = self.dropout(x)
        
        # Возвращаемся к исходной размерности
        x = x.permute(1, 0, 2)  # (batch_size, window_size, hidden_size)
        
        # Усреднение по временной оси
        x = x.mean(dim=1)  # (batch_size, hidden_size)
        
        # Выходной слой
        x = self.output_proj(x)  # (batch_size, num_classes)
        
        return x

In [20]:
# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [21]:
# Загрузим лучшие модели
input_size = 109  # Количество фичей
hidden_size = 64  # Размер скрытого состояния
num_layers = 2  # Количество слоев
num_classes = 3  # Количество классов

TS_attention_model = TSmixer(input_size, hidden_size, num_layers, num_classes)
TS_attention_model.load_state_dict(torch.load('best_nn/TS_attention_model_2_val_best_acc.pth'))
TS_attention_model.to(device)
TS_attention_model.eval()

TSmixer(
  (input_proj): Linear(in_features=109, out_features=64, bias=True)
  (conv_layers): ModuleList(
    (0-1): 2 x Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  )
  (attention_layers): ModuleList(
    (0-1): 2 x MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
  )
  (output_proj): Linear(in_features=64, out_features=3, bias=True)
  (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [22]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dropout_rate=0.5):
        super(BasicBlock, self).__init__()
        # Первый Conv1d слой
        self.conv1 = nn.Conv1d(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=kernel_size // 2,  # Сохраняет размерность
            bias=False
        )
        self.bn1 = nn.BatchNorm1d(out_channels)  # BatchNorm
        self.relu = nn.ReLU(inplace=True)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Второй Conv1d слой
        self.conv2 = nn.Conv1d(
            in_channels=out_channels,
            out_channels=out_channels,
            kernel_size=kernel_size,
            stride=1,
            padding=kernel_size // 2,
            bias=False
        )
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Остаточное соединение
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        residual = x

        # Первый Conv1d + BatchNorm + ReLU
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout1(x)

        # Второй Conv1d + BatchNorm
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        # Остаточное соединение
        residual = self.shortcut(residual)
        x += residual

        # ReLU после сложения
        x = self.relu(x)

        return x


class ResNet1D(nn.Module):
    def __init__(self, input_shape, num_classes, num_blocks=[2, 2, 2], initial_channels=64, dropout_rate=0.5):
        super(ResNet1D, self).__init__()
        seq_len, num_features = input_shape  # [Seq_len, Features]

        # Начальный слой
        self.initial_conv = nn.Sequential(
            nn.Conv1d(
                in_channels=num_features,
                out_channels=initial_channels,
                kernel_size=7,
                stride=2,
                padding=3,
                bias=False
            ),
            nn.BatchNorm1d(initial_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        )

        # Residual Blocks
        self.layer1 = self._make_layer(initial_channels, initial_channels, num_blocks[0], stride=1, dropout_rate=dropout_rate)
        self.layer2 = self._make_layer(initial_channels, initial_channels * 2, num_blocks[1], stride=2, dropout_rate=dropout_rate)
        self.layer3 = self._make_layer(initial_channels * 2, initial_channels * 4, num_blocks[2], stride=2, dropout_rate=dropout_rate)

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)

        # Классификатор
        self.fc = nn.Linear(initial_channels * 4, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride, dropout_rate):
        layers = []
        # Первый блок с изменением размерности
        layers.append(BasicBlock(in_channels, out_channels, stride=stride, dropout_rate=dropout_rate))
        # Остальные блоки
        for _ in range(1, num_blocks):
            layers.append(BasicBlock(out_channels, out_channels, stride=1, dropout_rate=dropout_rate))
        return nn.Sequential(*layers)

    def forward(self, x):
        # x shape: [Batch, Seq_len, Features]
        x = x.permute(0, 2, 1)  # [Batch, Features, Seq_len]

        # Начальный слой
        x = self.initial_conv(x)  # [Batch, initial_channels, Seq_len / 4]

        # Residual Blocks
        x = self.layer1(x)  # [Batch, initial_channels, Seq_len / 4]
        x = self.layer2(x)  # [Batch, initial_channels * 2, Seq_len / 8]
        x = self.layer3(x)  # [Batch, initial_channels * 4, Seq_len / 16]

        # Global Average Pooling
        x = self.global_avg_pool(x)  # [Batch, initial_channels * 4, 1]
        x = x.squeeze(-1)  # [Batch, initial_channels * 4]

        # Классификация
        x = self.fc(x)  # [Batch, num_classes]

        return x

In [23]:
input_shape = (30, 109)
num_classes = 3  # Количество классов
num_blocks = [3, 3, 3]  # Количество блоков в каждом слое
initial_channels = 64  # Начальное количество каналов

res_net_model_tr_2 = ResNet1D(input_shape=input_shape, num_classes=num_classes,num_blocks=num_blocks, initial_channels=initial_channels)
res_net_model_tr_2.load_state_dict(torch.load('best_nn/resnet_1d_2_train_best_acc.pth'))
res_net_model_tr_2.to(device)
res_net_model_tr_2.eval()

ResNet1D(
  (initial_conv): Sequential(
    (0): Conv1d(109, 64, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (conv2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout2): Dropout(p=0.5, inplace=False)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=F

In [24]:
num_blocks = [3, 3, 3] # Количество блоков в каждом слое
initial_channels = 128  # Начальное количество каналов

res_net_model_tr_3 = ResNet1D(input_shape=input_shape, num_classes=num_classes,num_blocks=num_blocks, initial_channels=initial_channels)
res_net_model_tr_3.load_state_dict(torch.load('best_nn/resnet_1d_3_train_best_acc.pth'))
res_net_model_tr_3.to(device)
res_net_model_tr_3.eval()

ResNet1D(
  (initial_conv): Sequential(
    (0): Conv1d(109, 128, kernel_size=(7,), stride=(2,), padding=(3,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool1d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (dropout1): Dropout(p=0.5, inplace=False)
      (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dropout2): Dropout(p=0.5, inplace=False)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1

In [25]:
scaler_ML = load_scaler('scaler_ML.pkl')
scaler_NN = load_scaler('scaler_nn.pkl')

# Предсказания моделей ML

In [26]:
eth_df_ML = eth_df_features.copy()

In [27]:
eth_df_ML

,Close,High,Low,Open,Volume,btc_close,Close_lag_3,High_lag_3,Low_lag_3,Open_lag_3,Volume_lag_3,btc_close_lag_3,Close_lag_9,High_lag_9,Low_lag_9,Open_lag_9,Volume_lag_9,btc_close_lag_9,Close_lag_15,High_lag_15,Low_lag_15,Open_lag_15,Volume_lag_15,btc_close_lag_15,Close_lag_30,High_lag_30,Low_lag_30,Open_lag_30,Volume_lag_30,btc_close_lag_30,Close_SMA_6,Close_SMA_9,Close_SMA_30,Close_SMA_50,Close_RSI_4,Close_RSI_9,Close_RSI_20,Close_mean_3,Close_median_3,Close_min_3,Close_max_3,Close_std_3,Close_range_3,Close_mean_10,Close_median_10,Close_min_10,Close_max_10,Close_std_10,Close_range_10,Close_mean_30,Close_median_30,Close_min_30,Close_max_30,Close_std_30,Close_range_30,Close_mean_50,Close_median_50,Close_min_50,Close_max_50,Close_std_50,Close_range_50,btc_close_mean_3,btc_close_median_3,btc_close_min_3,btc_close_max_3,btc_close_std_3,btc_close_range_3,btc_close_mean_10,btc_close_median_10,btc_close_min_10,btc_close_max_10,btc_close_std_10,btc_close_range_10,btc_close_mean_30,btc_close_median_30,btc_close_min_30,btc_close_max_30,btc_close_std_30,btc_close_range_30,btc_close_mean_50,btc_close_median_50,btc_close_min_50,btc_close_max_50,btc_close_std_50,btc_close_range_50,High-Low,Close_ratio_1,Close_log_diff_1,Close_momentum_3,Close_roc_3,Close_momentum_10,Close_roc_10,Close_momentum_30,Close_roc_30,Close_momentum_50,Close_roc_50,btc_close_ratio_1,btc_close_log_diff_1,btc_close_momentum_3,btc_close_roc_3,btc_close_momentum_10,btc_close_roc_10,btc_close_momentum_30,btc_close_roc_30,btc_close_momentum_50,btc_close_roc_50,Month,Weekday,Hour,target
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-01-01 12:30:00+00:00,3343.072754,3344.783691,3341.038330,3343.430420,4.343808e+07,93679.632812,3334.746826,3339.170166,3334.012695,3337.341064,4.343808e+07,93267.125000,3325.237549,3328.333740,3323.114502,3323.114502,9.007104e+06,93254.468750,3330.404053,3333.293213,3328.243164,3331.517090,5076992.0,93259.992188,3339.198975,3339.198975,3334.000000,3334.000000,1.054720e+06,93447.375000,3336.257935,3331.594265,3332.004256,3337.854551,87.426735,68.418219,57.020899,3340.554118,3343.072754,3335.316406,3343.273193,4.537098,7.956787,3330.958594,3330.569214,3319.422119,3343.273193,8.375223,23.851074,3332.004256,3334.248535,3312.978760,3343.273193,7.791334,30.294434,3337.854551,3338.296997,3312.978760,3358.024658,10.136359,45.045898,93513.197917,93616.367188,93243.593750,93679.632812,235.617147,436.039062,93304.431250,93258.507812,93111.765625,93679.632812,195.346178,567.867188,93349.735156,93330.218750,92867.187500,93679.632812,214.242752,812.445312,93471.558125,93515.777344,92867.187500,94256.054688,255.901936,1388.867188,3.745361,0.999940,-0.000060,8.325928,0.249672,17.716309,0.532764,3.873779,0.116009,5.786865,0.173400,1.000676,0.000676,412.507812,0.442286,442.554688,0.474655,232.257812,0.248544,348.664062,0.373578,1,2,12,0
2025-01-01 12:45:00+00:00,3340.229736,3340.972900,3338.161133,3340.972900,4.343808e+07,93618.367188,3335.316406,3335.985107,3333.931885,3334.479492,4.343808e+07,93243.593750,3324.588379,3329.039795,3323.121826,3325.411865,9.007104e+06,93138.328125,3325.482666,3333.349854,3325.310059,3330.390137,191223808.0,93106.484375,3342.497070,3342.497070,3338.213135,3338.907959,9.932800e+06,93582.289062,3338.470459,3333.332194,3331.928678,3337.841802,67.214291,62.324199,54.907598,3342.191895,3343.072754,3340.229736,3343.273193,1.702232,3.043457,3332.457812,3334.465332,3319.422119,3343.273193,8.576754,23.851074,3331.928678,3334.248535,3312.978760,3343.273193,7.696461,30.294434,3337.841802,3338.296997,3312.978760,3358.024658,10.132893,45.045898,93638.122396,93618.367188,93616.367188,93679.632812,35.962981,63.265625,93340.821094,93264.835938,93111.765625,93679.632812,217.628245,567.867188,93350.937760,93330.218750,92867.187500,93679.632812,215.689528,812.445312,93473.236250,93515.777344,92867.187500,94256.054688,256.597023,1388.867188,2.811768,0.999150,-0.000851,4.91333

In [28]:
val_size = 400  # Размер валидационной выборки
test_size = 400  # Размер тестовой выборки

# Валидационная выборка
val_df = eth_df_ML.iloc[-(val_size + test_size):-test_size]

# Тестовая выборка
test_df = eth_df_ML.iloc[-test_size:]

# Обучающая выборка (все остальные данные)
train_df = eth_df_ML.iloc[:-(val_size + test_size)]

# Проверка размеров выборок
print(f"Размер обучающей выборки (train): {len(train_df)}")
print(f"Размер валидационной выборки (val): {len(val_df)}")
print(f"Размер тестовой выборки (test): {len(test_df)}")

print("Количество каждого класса в обучающей выборке:")
print(train_df['target'].value_counts())

print("Количество каждого класса в валидационной выборке:")
print(val_df['target'].value_counts())

print("Количество каждого класса в тестовой выборке:")
print(test_df['target'].value_counts())

Размер обучающей выборки (train): 3939
Размер валидационной выборки (val): 400
Размер тестовой выборки (test): 400
Количество каждого класса в обучающей выборке:
target
0    1399
2    1369
1    1171
Name: count, dtype: int64
Количество каждого класса в валидационной выборке:
target
2    134
1    134
0    132
Name: count, dtype: int64
Количество каждого класса в тестовой выборке:
target
2    151
0    135
1    114
Name: count, dtype: int64


In [29]:
# Разделяем train_df
X_train = train_df.iloc[:, :-1]  # Все столбцы, кроме последнего
y_train = train_df.iloc[:, -1]   # Последний столбец (target)

# Разделяем val_df
X_val = val_df.iloc[:, :-1]  # Все столбцы, кроме последнего
y_val = val_df.iloc[:, -1]   # Последний столбец (target)

# Разделяем test_df
X_test = test_df.iloc[:, :-1]  # Все столбцы, кроме последнего
y_test = test_df.iloc[:, -1]   # Последний столбец (target)

# Проверка размеров
print(f"Размеры train: X_train = {X_train.shape}, y_train = {y_train.shape}")
print(f"Размеры val: X_val = {X_val.shape}, y_val = {y_val.shape}")
print(f"Размеры test: X_test = {X_test.shape}, y_test = {y_test.shape}")

# Вывод первых строк X_train и y_train
print("\nX_train:")
print(X_train.head())

print("\ny_train:")
print(y_train.head())

Размеры train: X_train = (3939, 109), y_train = (3939,)
Размеры val: X_val = (400, 109), y_val = (400,)
Размеры test: X_test = (400, 109), y_test = (400,)

X_train:
                                 Close         High          Low         Open  \
Datetime                                                                        
2025-01-01 12:30:00+00:00  3343.072754  3344.783691  3341.038330  3343.430420   
2025-01-01 12:45:00+00:00  3340.229736  3340.972900  3338.161133  3340.972900   
2025-01-01 13:00:00+00:00  3332.834473  3344.976318  3332.834473  3340.183594   
2025-01-01 13:15:00+00:00  3331.148926  3334.383545  3327.366943  3331.708984   
2025-01-01 13:30:00+00:00  3330.831055  3335.455566  3330.733887  3331.089355   

                               Volume     btc_close  Close_lag_3   High_lag_3  \
Datetime                                                                        
2025-01-01 12:30:00+00:00  43438080.0  93679.632812  3334.746826  3339.170166   
2025-01-01 12:45:00+00:0

In [30]:
# Предсказание на обучающей, валидационной и тестовой выборках
y_train_pred_xgboost = best_model_xgboost.predict(X_train)
y_val_pred_xgboost = best_model_xgboost.predict(X_val)
y_test_pred_xgboost = best_model_xgboost.predict(X_test)

# Предсказание вероятностей
y_train_prob_xgboost = best_model_xgboost.predict_proba(X_train)
y_val_prob_xgboost = best_model_xgboost.predict_proba(X_val)
y_test_prob_xgboost = best_model_xgboost.predict_proba(X_test)

In [31]:
# Предсказание на обучающей, валидационной и тестовой выборках
y_train_pred_LGBMC = best_model_LGBMC.predict(X_train)
y_val_pred_LGBMC = best_model_LGBMC.predict(X_val)
y_test_pred_LGBMC = best_model_LGBMC.predict(X_test)

# Предсказание вероятностей
y_train_prob_LGBMC = best_model_LGBMC.predict_proba(X_train)
y_val_prob_LGBMC = best_model_LGBMC.predict_proba(X_val)
y_test_prob_LGBMC = best_model_LGBMC.predict_proba(X_test)

In [32]:
print(y_train_pred_xgboost.shape, y_val_pred_xgboost.shape, y_test_pred_xgboost.shape)
print(y_train_prob_xgboost.shape, y_val_prob_xgboost.shape, y_test_prob_xgboost.shape)

print(y_train_pred_LGBMC.shape, y_val_pred_LGBMC.shape, y_test_pred_LGBMC.shape)
print(y_train_prob_LGBMC.shape, y_val_prob_LGBMC.shape, y_test_prob_LGBMC.shape)

(3939,) (400,) (400,)
(3939, 3) (400, 3) (400, 3)
(3939,) (400,) (400,)
(3939, 3) (400, 3) (400, 3)


# Предсказания моделей NN

In [33]:
# Валидационная выборка
val_df = eth_df_features.iloc[-(val_size + test_size):-test_size]

# Тестовая выборка
test_df = eth_df_features.iloc[-test_size:]

# Обучающая выборка (все остальные данные)
train_df = eth_df_features.iloc[:-(val_size + test_size)]

# Проверка размеров выборок
print(f"Размер обучающей выборки (train): {len(train_df)}")
print(f"Размер валидационной выборки (val): {len(val_df)}")
print(f"Размер тестовой выборки (test): {len(test_df)}")

print("Количество каждого класса в обучающей выборке:")
print(train_df['target'].value_counts())

print("Количество каждого класса в валидационной выборке:")
print(val_df['target'].value_counts())

print("Количество каждого класса в тестовой выборке:")
print(test_df['target'].value_counts())

Размер обучающей выборки (train): 3939
Размер валидационной выборки (val): 400
Размер тестовой выборки (test): 400
Количество каждого класса в обучающей выборке:
target
0    1399
2    1369
1    1171
Name: count, dtype: int64
Количество каждого класса в валидационной выборке:
target
2    134
1    134
0    132
Name: count, dtype: int64
Количество каждого класса в тестовой выборке:
target
2    151
0    135
1    114
Name: count, dtype: int64


In [34]:
# Разделяем данные на фичи и целевую переменную
features_train = train_df.drop(columns=['target'])
target_train = train_df['target']

features_val = val_df.drop(columns=['target'])
target_val = val_df['target']

features_test = test_df.drop(columns=['target'])
target_test = test_df['target']

In [35]:
# нормализация X
X_train_scaled, _ = normalize_dataframe(features_train, scaler=scaler_NN, save_scaler=False)

# Применение того же scaler к валидационной и тестовой выборкам
X_val_scaled, _ = normalize_dataframe(features_val, scaler=scaler_NN, save_scaler=False)
X_test_scaled, _ = normalize_dataframe(features_test, scaler=scaler_NN, save_scaler=False)

In [36]:
# Создаем новые датафреймы с нормализованными данными
train_scaled_df = pd.DataFrame(X_train_scaled, columns=features_train.columns, index=features_train.index)
val_scaled_df = pd.DataFrame(X_val_scaled, columns=features_val.columns, index=features_val.index)
test_scaled_df = pd.DataFrame(X_test_scaled, columns=features_test.columns, index=features_test.index)

In [37]:
window_size = 30

# создаем окна для трейдинга
def create_windows_for_predict(data, window_size):
    X = []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)  # Окно фичей
    return np.array(X)

In [38]:
X_train = create_windows_for_predict(train_scaled_df, window_size=window_size)
X_val = create_windows_for_predict(val_scaled_df, window_size=window_size)
X_test = create_windows_for_predict(test_scaled_df, window_size=window_size)

In [39]:
# Преобразование в тензоры PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)

In [40]:
# Проверяем размерности
print(X_train_tensor.shape, X_val_tensor.shape, X_test_tensor.shape)

torch.Size([3909, 30, 109]) torch.Size([370, 30, 109]) torch.Size([370, 30, 109])


In [41]:
def get_prediction(model, tensor):
    # Проверяем, что модель в режиме оценки
    model.eval()

	# Делаем предсказания
    with torch.no_grad():
        outputs = model(tensor)
        probabilities = torch.softmax(outputs, dim=1)  # Преобразуем в вероятности
        predicted_classes = torch.argmax(outputs, dim=1)  # Предсказанные классы

    return predicted_classes.cpu().numpy(), probabilities.cpu().numpy()


In [42]:
y_train_pred_TS_attention, y_train_prob_TS_attention = get_prediction(TS_attention_model, X_train_tensor)
y_val_pred_TS_attention, y_val_prob_TS_attention = get_prediction(TS_attention_model, X_val_tensor)
y_test_pred_TS_attention, y_test_prob_TS_attention = get_prediction(TS_attention_model, X_test_tensor)

y_train_pred_res_net_2, y_train_prob_res_net_2 = get_prediction(res_net_model_tr_2, X_train_tensor)
y_val_pred_res_net_2, y_val_prob_res_net_2 = get_prediction(res_net_model_tr_2, X_val_tensor)
y_test_pred_res_net_2, y_test_prob_res_net_2 = get_prediction(res_net_model_tr_2, X_test_tensor)

y_train_pred_res_net_3, y_train_prob_res_net_3 = get_prediction(res_net_model_tr_3, X_train_tensor)
y_val_pred_res_net_3, y_val_prob_res_net_3 = get_prediction(res_net_model_tr_3, X_val_tensor)
y_test_pred_res_net_3, y_test_prob_res_net_3 = get_prediction(res_net_model_tr_3, X_test_tensor)

In [43]:
print(y_train_pred_TS_attention.shape, y_train_pred_res_net_2.shape, y_train_pred_res_net_3.shape)
print(y_val_pred_TS_attention.shape, y_val_pred_res_net_2.shape, y_val_pred_res_net_3.shape)
print(y_test_pred_TS_attention.shape, y_test_pred_res_net_2.shape, y_test_pred_res_net_3.shape)

print(y_train_prob_TS_attention.shape, y_train_prob_res_net_2.shape, y_train_prob_res_net_3.shape)
print(y_val_prob_TS_attention.shape, y_val_prob_res_net_2.shape, y_val_prob_res_net_3.shape)
print(y_test_prob_TS_attention.shape, y_test_prob_res_net_2.shape, y_test_prob_res_net_3.shape)

(3909,) (3909,) (3909,)
(370,) (370,) (370,)
(370,) (370,) (370,)
(3909, 3) (3909, 3) (3909, 3)
(370, 3) (370, 3) (370, 3)
(370, 3) (370, 3) (370, 3)


# Подготовка данных для ансамбля

In [44]:
# Объединим предсказания моделей в датасеты

# Обрезаем первые 30 строк для XGBoost и LightGBM
y_train_pred_xgboost = y_train_pred_xgboost[30:]
y_val_pred_xgboost = y_val_pred_xgboost[30:]
y_test_pred_xgboost = y_test_pred_xgboost[30:]

y_train_prob_xgboost = y_train_prob_xgboost[30:]
y_val_prob_xgboost = y_val_prob_xgboost[30:]
y_test_prob_xgboost = y_test_prob_xgboost[30:]

y_train_pred_LGBMC = y_train_pred_LGBMC[30:]
y_val_pred_LGBMC = y_val_pred_LGBMC[30:]
y_test_pred_LGBMC = y_test_pred_LGBMC[30:]

y_train_prob_LGBMC = y_train_prob_LGBMC[30:]
y_val_prob_LGBMC = y_val_prob_LGBMC[30:]
y_test_prob_LGBMC = y_test_prob_LGBMC[30:]

# Создаём DataFrame для train
train_ensemble_df = pd.DataFrame(index=train_df.index[30:])
train_ensemble_df['y_true'] = y_train[30:]

# Добавляем предсказания XGBoost
train_ensemble_df['xgboost_pred'] = y_train_pred_xgboost
train_ensemble_df['xgboost_prob_class_0'] = y_train_prob_xgboost[:, 0]
train_ensemble_df['xgboost_prob_class_1'] = y_train_prob_xgboost[:, 1]
train_ensemble_df['xgboost_prob_class_2'] = y_train_prob_xgboost[:, 2]

# Добавляем предсказания LightGBM
train_ensemble_df['lightgbm_pred'] = y_train_pred_LGBMC
train_ensemble_df['lightgbm_prob_class_0'] = y_train_prob_LGBMC[:, 0]
train_ensemble_df['lightgbm_prob_class_1'] = y_train_prob_LGBMC[:, 1]
train_ensemble_df['lightgbm_prob_class_2'] = y_train_prob_LGBMC[:, 2]

# Добавляем предсказания TS Attention
train_ensemble_df['ts_attention_pred'] = y_train_pred_TS_attention
train_ensemble_df['ts_attention_prob_class_0'] = y_train_prob_TS_attention[:, 0]
train_ensemble_df['ts_attention_prob_class_1'] = y_train_prob_TS_attention[:, 1]
train_ensemble_df['ts_attention_prob_class_2'] = y_train_prob_TS_attention[:, 2]

# Добавляем предсказания ResNet 2
train_ensemble_df['resnet_2_pred'] = y_train_pred_res_net_2
train_ensemble_df['resnet_2_prob_class_0'] = y_train_prob_res_net_2[:, 0]
train_ensemble_df['resnet_2_prob_class_1'] = y_train_prob_res_net_2[:, 1]
train_ensemble_df['resnet_2_prob_class_2'] = y_train_prob_res_net_2[:, 2]

# Добавляем предсказания ResNet 3
train_ensemble_df['resnet_3_pred'] = y_train_pred_res_net_3
train_ensemble_df['resnet_3_prob_class_0'] = y_train_prob_res_net_3[:, 0]
train_ensemble_df['resnet_3_prob_class_1'] = y_train_prob_res_net_3[:, 1]
train_ensemble_df['resnet_3_prob_class_2'] = y_train_prob_res_net_3[:, 2]

# Аналогично создаём DataFrame для val и test
val_ensemble_df = pd.DataFrame(index=val_df.index[30:])
val_ensemble_df['y_true'] = y_val[30:]

val_ensemble_df['xgboost_pred'] = y_val_pred_xgboost
val_ensemble_df['xgboost_prob_class_0'] = y_val_prob_xgboost[:, 0]
val_ensemble_df['xgboost_prob_class_1'] = y_val_prob_xgboost[:, 1]
val_ensemble_df['xgboost_prob_class_2'] = y_val_prob_xgboost[:, 2]

val_ensemble_df['lightgbm_pred'] = y_val_pred_LGBMC
val_ensemble_df['lightgbm_prob_class_0'] = y_val_prob_LGBMC[:, 0]
val_ensemble_df['lightgbm_prob_class_1'] = y_val_prob_LGBMC[:, 1]
val_ensemble_df['lightgbm_prob_class_2'] = y_val_prob_LGBMC[:, 2]

val_ensemble_df['ts_attention_pred'] = y_val_pred_TS_attention
val_ensemble_df['ts_attention_prob_class_0'] = y_val_prob_TS_attention[:, 0]
val_ensemble_df['ts_attention_prob_class_1'] = y_val_prob_TS_attention[:, 1]
val_ensemble_df['ts_attention_prob_class_2'] = y_val_prob_TS_attention[:, 2]

val_ensemble_df['resnet_2_pred'] = y_val_pred_res_net_2
val_ensemble_df['resnet_2_prob_class_0'] = y_val_prob_res_net_2[:, 0]
val_ensemble_df['resnet_2_prob_class_1'] = y_val_prob_res_net_2[:, 1]
val_ensemble_df['resnet_2_prob_class_2'] = y_val_prob_res_net_2[:, 2]

val_ensemble_df['resnet_3_pred'] = y_val_pred_res_net_3
val_ensemble_df['resnet_3_prob_class_0'] = y_val_prob_res_net_3[:, 0]
val_ensemble_df['resnet_3_prob_class_1'] = y_val_prob_res_net_3[:, 1]
val_ensemble_df['resnet_3_prob_class_2'] = y_val_prob_res_net_3[:, 2]

test_ensemble_df = pd.DataFrame(index=test_df.index[30:])
test_ensemble_df['y_true'] = y_test[30:]

test_ensemble_df['xgboost_pred'] = y_test_pred_xgboost
test_ensemble_df['xgboost_prob_class_0'] = y_test_prob_xgboost[:, 0]
test_ensemble_df['xgboost_prob_class_1'] = y_test_prob_xgboost[:, 1]
test_ensemble_df['xgboost_prob_class_2'] = y_test_prob_xgboost[:, 2]

test_ensemble_df['lightgbm_pred'] = y_test_pred_LGBMC
test_ensemble_df['lightgbm_prob_class_0'] = y_test_prob_LGBMC[:, 0]
test_ensemble_df['lightgbm_prob_class_1'] = y_test_prob_LGBMC[:, 1]
test_ensemble_df['lightgbm_prob_class_2'] = y_test_prob_LGBMC[:, 2]

test_ensemble_df['ts_attention_pred'] = y_test_pred_TS_attention
test_ensemble_df['ts_attention_prob_class_0'] = y_test_prob_TS_attention[:, 0]
test_ensemble_df['ts_attention_prob_class_1'] = y_test_prob_TS_attention[:, 1]
test_ensemble_df['ts_attention_prob_class_2'] = y_test_prob_TS_attention[:, 2]

test_ensemble_df['resnet_2_pred'] = y_test_pred_res_net_2
test_ensemble_df['resnet_2_prob_class_0'] = y_test_prob_res_net_2[:, 0]
test_ensemble_df['resnet_2_prob_class_1'] = y_test_prob_res_net_2[:, 1]
test_ensemble_df['resnet_2_prob_class_2'] = y_test_prob_res_net_2[:, 2]

test_ensemble_df['resnet_3_pred'] = y_test_pred_res_net_3
test_ensemble_df['resnet_3_prob_class_0'] = y_test_prob_res_net_3[:, 0]
test_ensemble_df['resnet_3_prob_class_1'] = y_test_prob_res_net_3[:, 1]
test_ensemble_df['resnet_3_prob_class_2'] = y_test_prob_res_net_3[:, 2]

In [45]:
columns_prob = ['xgboost_prob_class_0', 'xgboost_prob_class_1', 'xgboost_prob_class_2', 
                'lightgbm_prob_class_0', 'lightgbm_prob_class_1', 'lightgbm_prob_class_2',
                'ts_attention_prob_class_0', 'ts_attention_prob_class_1', 'ts_attention_prob_class_2',
                'resnet_2_prob_class_0', 'resnet_2_prob_class_1', 'resnet_2_prob_class_2,'
                'resnet_3_prob_class_0', 'resnet_3_prob_class_1', 'resnet_3_prob_class_2']

columns_class_0 = ['xgboost_prob_class_0', 'lightgbm_prob_class_0', 'ts_attention_prob_class_0', 'resnet_2_prob_class_0', 'resnet_3_prob_class_0']
columns_class_1 = ['xgboost_prob_class_1', 'lightgbm_prob_class_1', 'ts_attention_prob_class_1', 'resnet_2_prob_class_1', 'resnet_3_prob_class_1']
columns_class_2 = ['xgboost_prob_class_2', 'lightgbm_prob_class_2', 'ts_attention_prob_class_2', 'resnet_2_prob_class_2', 'resnet_3_prob_class_2']

columns_pred = ['xgboost_pred', 'lightgbm_pred', 'ts_attention_pred', 'resnet_2_pred', 'resnet_3_pred']

columns = list(train_ensemble_df.drop(['y_true'], axis=1).reset_index(drop=True).columns)
columns

['xgboost_pred',
 'xgboost_prob_class_0',
 'xgboost_prob_class_1',
 'xgboost_prob_class_2',
 'lightgbm_pred',
 'lightgbm_prob_class_0',
 'lightgbm_prob_class_1',
 'lightgbm_prob_class_2',
 'ts_attention_pred',
 'ts_attention_prob_class_0',
 'ts_attention_prob_class_1',
 'ts_attention_prob_class_2',
 'resnet_2_pred',
 'resnet_2_prob_class_0',
 'resnet_2_prob_class_1',
 'resnet_2_prob_class_2',
 'resnet_3_pred',
 'resnet_3_prob_class_0',
 'resnet_3_prob_class_1',
 'resnet_3_prob_class_2']

In [46]:
train_ensemble_df

,y_true,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-01-01 20:00:00+00:00,1,1,0.124778,0.785694,0.089528,1,0.285671,0.376139,0.338190,1,0.312960,0.428118,0.258922,0,0.980260,0.018403,0.001337,0,0.986632,0.012389,0.000980
2025-01-01 20:15:00+00:00,2,2,0.027008,0.196353,0.776639,2,0.252593,0.332584,0.414823,1,0.314431,0.426569,0.259000,0,0.982533,0.016351,0.001116,0,0.989017,0.010224,0.000759
2025-01-01 20:30:00+00:00,2,2,0.042673,0.168716,0.788611,2,0.283406,0.335966,0.380627,1,0.311271,0.432529,0.256200,0,0.981677,0.017150,0.001173,0,0.988274,0.010881,0.000845
2025-01-01 20:45:00+00:00,1,1,0.038654,0.835247,0.126099,1,0.236530,0.408140,0.355331,1,0.309958,0.435798,0.254244,0,0.981230,0.017571,0.001199,0,0.985484,0.013271,0.001245
2025-01-01 21:00:00+00:00,2,2,0.022424,0.249628,0.727948,2,0.250620,0.372881,0.376499,1,0.310040,0.437117,0.252843,0,0.981720,0.017098,0.001182,0,0.987526,0.011498,0.000976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-11 13:45:00+00:00,2,2,0.075006,0.029190,0.895804,2,0.288225,0.233299,0.478477,0,0.368701,0.295742,0.335557,0,0.424434,0.354351,0.221215,0,0.876050,0.101343,0.022607
2025-02-11 14:00:00+00:00,2,2,0.130394,0.025571,0.844034,2,0.347025,0.232258,0.420717,2,0.338784,0.212372,0.448843,2,0.087086,0.223869,0.689045,2,0.170355,0.342048,0.487598
2025-02-11 14:15:00+00:00,1,1,0.108009,0.743309,0.148681,2,0.333471,0.262245,0.404284,2,0.372521,0.252054,0.375426,2,0.164273,0.344255,0.491472,1,0.380581,0.404932,0.214488


In [47]:
val_ensemble_df

,y_true,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-02-11 22:30:00+00:00,0,2,0.199069,0.179576,0.621355,2,0.303377,0.241512,0.455112,2,0.350907,0.238517,0.410576,2,0.006459,0.002723,0.990818,2,0.024836,0.007366,0.967797
2025-02-11 22:45:00+00:00,2,2,0.074061,0.235702,0.690238,2,0.309384,0.226492,0.464124,2,0.350148,0.220224,0.429628,2,0.003630,0.000501,0.995869,2,0.012871,0.001970,0.985159
2025-02-11 23:00:00+00:00,2,2,0.105230,0.220809,0.673961,2,0.290649,0.212776,0.496575,2,0.340427,0.203829,0.455744,2,0.002204,0.000169,0.997627,2,0.005273,0.000550,0.994177
2025-02-11 23:15:00+00:00,1,2,0.064104,0.077304,0.858592,2,0.331677,0.222120,0.446203,2,0.349512,0.215900,0.434588,2,0.002026,0.000340,0.997634,2,0.007415,0.001121,0.991464
2025-02-11 23:30:00+00:00,1,2,0.028422,0.090304,0.881274,2,0.310522,0.239720,0.449757,2,0.349801,0.212694,0.437505,2,0.002573,0.000645,0.996782,2,0.005300,0.002055,0.992646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-15 17:45:00+00:00,2,0,0.585301,0.248171,0.166528,0,0.395607,0.316527,0.287866,1,0.318440,0.396575,0.284985,1,0.325759,0.421385,0.252856,1,0.156938,0.446344,0.396718
2025-02-15 18:00:00+00:00,2,0,0.442781,0.239947,0.317272,0,0.401615,0.271228,0.327158,1,0.318058,0.396921,0.285021,1,0.314738,0.423795,0.261467,2,0.126906,0.433410,0.439684
2025-02-15 18:15:00+00:00,2,0,0.504354,0.239085,0.256561,0,0.394383,0.282892,0.322725,1,0.318558,0.396011,0.285431,1,0.338310,0.432383,0.229307,1,0.186763,0.500486,0.312751


In [48]:
test_ensemble_df

,y_true,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2
Datetime,,,,,,,,,,,,,,,,,,,,,
2025-02-16 02:30:00+00:00,1,0,0.380651,0.338186,0.281163,0,0.400347,0.280106,0.319547,1,0.322676,0.424026,0.253298,0,0.516098,0.365405,0.118497,0,0.575365,0.341983,0.082653
2025-02-16 02:45:00+00:00,2,0,0.510216,0.245643,0.244141,0,0.401615,0.271228,0.327158,1,0.321754,0.425027,0.253219,1,0.363029,0.432404,0.204567,0,0.586391,0.326251,0.087358
2025-02-16 03:00:00+00:00,2,1,0.388277,0.396003,0.215720,0,0.401615,0.271228,0.327158,1,0.318286,0.425068,0.256646,0,0.470212,0.384098,0.145690,0,0.650011,0.281789,0.068200
2025-02-16 03:15:00+00:00,2,0,0.586930,0.285569,0.127501,0,0.372023,0.324924,0.303052,1,0.316071,0.427713,0.256216,1,0.403143,0.416276,0.180581,0,0.675739,0.260325,0.063935
2025-02-16 03:30:00+00:00,2,2,0.405403,0.174637,0.419960,0,0.401615,0.271228,0.327158,1,0.312343,0.430697,0.256960,0,0.511656,0.352612,0.135732,0,0.590386,0.324001,0.085613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-19 21:45:00+00:00,2,0,0.461258,0.280298,0.258444,0,0.357861,0.312984,0.329156,2,0.329736,0.085076,0.585188,1,0.404056,0.424781,0.171164,0,0.739210,0.213921,0.046869
2025-02-19 22:00:00+00:00,2,1,0.294153,0.452905,0.252942,0,0.357861,0.312984,0.329156,2,0.356485,0.107799,0.535716,1,0.270490,0.449912,0.279597,0,0.617360,0.298960,0.083680
2025-02-19 22:15:00+00:00,2,1,0.274114,0.467359,0.258527,0,0.379454,0.302459,0.318087,2,0.392250,0.211248,0.396502,1,0.383239,0.428192,0.188568,0,0.733649,0.211121,0.055230


# Загрузим данные для трейдинга

In [49]:
# Напишем свой бэктестинг стратегий
class StrategyTester:
    def __init__(self, df, initial_balance=100000, position_size=1.0, commission=0.1, stop_loss=0.0, trailing_stop=0.0, period_hold_position=0, reverse=True):
        """
        Инициализация тестировщика стратегии.

        :param df: DataFrame с колонками ['Close', 'High', 'Low', 'Open', 'Volume', 'Signal', 'SMA' (опционально)].
        :param initial_balance: Начальный баланс.
        :param position_size: Размер позиции в долях (1.0 = 100% от капитала).
        :param commission: Комиссия в процентах.
        :param stop_loss: Стоп-лосс в процентах. Если 0.0, то не используется.
        :param trailing_stop: Трейлинг-стоп в процентах. Если 0.0, то не используется.
        :param period_hold_position: Сколько периодов удердиваем позицию. Если 0 - то проверяем открытие/закрытие позиций каждый период.
        Если например 2, то после взода два периода удерживаем сделку, потом проверяем сигналы.

        :param reverse: Если True, то сделка закрывается только когда приходит противоположный сигнал.
        Если False, то сделка закрывается при любом изменении сигнала
        """
        self.df = df.copy()
        self.initial_balance = initial_balance
        self.position_size = position_size
        self.commission = commission / 100 # Переводим % в доли
        self.df['Drawdown'] = 0 # Добавляем колонку для Drawdown
        self.df['Equity'] = initial_balance # Добавляем колонку для Equity

        self.equity = initial_balance # Текущий эквити
        self.positions = []  # Хранит открытые сделки
        self.trades = []  # Хранит закрытые сделки
        self.cur_position = 0 # Текущая позиция (1 - длинная, -1 - короткая, 0 - нет позиции)
        self.pos_price_open = 0.0 # Цена открытия позиции

        self.reverse = reverse # Стратегия ревесная или нет

        self.period_hold_position = period_hold_position # Какое количество периодов удерживаем позицию. 0 - проверяем каждый период
        self.count_period_for_hold = 0 # для отслеживания периодов удержания позиций

        self.stop_loss = stop_loss / 100 # Переводим % в доли
        self.trailing_stop = trailing_stop / 100 # Переводим % в доли
        self.stop_loss_price = 0.0 # Для отслеживания стоп-лосса
        self.trailing_stop_price = 0.0 # Для отслеживания трейлинг-стопа

        if stop_loss == 0.0 and trailing_stop > 0.0: # Если уствновлен трейлинг-стоп, то должен быть установлен и стоп-лосс для корректной работы
            self.stop_loss = self.trailing_stop
        
        # self.df_result = pd.DataFrame(columns=['type', 'entry', 'exit', 'pnl'])

    def run(self):
        """
        Проход по всем данным DataFrame и выполнение сделок.
        """
        for i in range(2, len(self.df)):
            # проверяем условие удержания позиции
            if self.period_hold_position != 0 and self.cur_position != 0 and i != len(self.df) - 1: # Если есть открытые позици и если позицию удерживаем и свеча не последняя
                if self.period_hold_position != self.count_period_for_hold: # если количество циклов удержания не прошли
                    self.count_period_for_hold += 1 # прибавляем один пройденный цикл удержания
                    # расчет equity и drowdown
                    eq_max = self.df['Equity'].max()
                    self.df.loc[self.df.index[i], 'Equity'] = self.equity
                    drowdown = (self.equity - eq_max) / eq_max
                    self.df.loc[self.df.index[i], 'Drawdown'] = drowdown * 100

                    continue # пропускаем цикл
                else:
                    self.count_period_for_hold = 0 # если все циклы удержания пройдены, обнуляем счетчик и выполняем основной цикл

            signal = self.df.loc[self.df.index[i - 1], 'Signal'] # Получаем текущий сигнал
            signal_prev = self.df.loc[self.df.index[i - 2], 'Signal'] # Получаем предыдущий сигнал
            cur_price = self.df.loc[self.df.index[i], 'Close'] # Получаем текущию цену

            # Проверка Стоп-лосса
            if self.stop_loss > 0.0: # Если стоп-лосс используется
                if self.cur_position == 1: # Если мы в покупке
                    if cur_price < self.stop_loss_price: # Если текущая цена стала меньше цены стоп-лосса, то закрываем покупку
                        self.close_trades(cur_price, self.df.index[i], stop_l=True)
                if self.cur_position == -1: # Если мы в продаже
                    if cur_price > self.stop_loss_price: # Если текущая цена стала больше цены стоп-лосса, то закрываем продажу
                        self.close_trades(cur_price, self.df.index[i], stop_l=True)

            # Расчет Трейлинг-стопа
            if self.trailing_stop > 0.0: # Если Трейлинг-стоп используется
                if self.cur_position == 1: # Если мы в покупке
                    if self.trailing_stop_price > 0.0: # Если цена трейлинг-стопа определена (мы уже подтягивали стоп)
                        # Если текущая цена прошла два расстояния равных трейлинг-стопу от цены стоп-лосса, то подтягиваем стоп-лосс
                        if cur_price >= self.trailing_stop_price + 2 * self.trailing_stop_price * self.trailing_stop:
                            self.stop_loss_price = cur_price * (1 - self.trailing_stop) # подтягиваем стоп-лосс на величину трейлинг-стопа от текущей цены
                            self.trailing_stop_price = self.stop_loss_price # Устанавливаем новую цену для трейлинг-стопа (значит что стоп мы уже подтянули)
                    else: # Если стоп еще не подтягивали, то проверяем и подтягиваем
                        if cur_price >= self.pos_price_open + self.pos_price_open * self.trailing_stop: # Если цена прошла вверх от точки входа на расстояние больше трейлинг-стопа
                            self.stop_loss_price = cur_price * (1 - self.trailing_stop) # подтягиваем стоп-лосс на величину трейлинг-стопа от текущей цены
                            self.trailing_stop_price = self.stop_loss_price # Устанавливаем цену для трейлинг-стопа (значит что стоп мы уже подтянули)

                if self.cur_position == -1: # Если мы в продаже
                    if self.trailing_stop_price > 0.0: # Если цена трейлинг-стопа определена (мы уже подтягивали стоп)
                        ## Если текущая цена прошла два расстояния равных трейлинг-стопу от цены стоп-лосса, то подтягиваем стоп-лосс
                        if cur_price <= self.trailing_stop_price - 2 * self.trailing_stop_price * self.trailing_stop:
                            self.stop_loss_price = cur_price * (1 + self.trailing_stop) # подтягиваем стоп-лосс на величину трейлинг-стопа от текущей цены
                            self.trailing_stop_price = self.stop_loss_price # Устанавливаем новую цену для трейлинг-стопа (снова подтягиваем)
                    else: # Если стоп еще не подтягивали, то проверяем и подтягиваем
                        if cur_price <= self.pos_price_open - self.pos_price_open * self.trailing_stop: # Если цена прошла вниз от точки входа на расстояние больше трейлинг-стопа
                            self.stop_loss_price = cur_price * (1 + self.trailing_stop) # подтягиваем стоп-лосс на величину трейлинг-стопа от текущей цены
                            self.trailing_stop_price = self.stop_loss_price # Устанавливаем цену для трейлинг-стопа (значит что стоп мы уже подтянули)

            # Открытие сделки
            if signal != signal_prev: # если сигнал изменился
                if signal == 1 and self.cur_position != 1:  # Сигнал на покупку
                    if self.cur_position == -1:
                        self.close_trades(cur_price, self.df.index[i])

                    self.open_trade('buy', cur_price, self.df.index[i]) # Открывем сделку на покупку

                    if self.stop_loss > 0.0: # Если стоп-лосс используется
                        self.stop_loss_price = cur_price * (1 - self.stop_loss) # Устанавливаем цену стоп-лосса для покупки ниже цены входа

                elif signal == -1 and self.cur_position != -1:  # Сигнал на продажу
                    if self.cur_position == 1:
                        self.close_trades(cur_price, self.df.index[i])
                        
                    self.open_trade('sell', cur_price, self.df.index[i]) # Открывем сделку на продажу

                    if self.stop_loss > 0.0: # Если стоп-лосс используется
                        self.stop_loss_price = cur_price * (1 + self.stop_loss) # Устанавливаем цену стоп-лосса для продажи выше цены входа

                else: # если пришел 0 сигнал
                    if not self.reverse: # Если стратегия не реверсная (покупка закрывается когда открывается продажа и наоборот)
                        if self.cur_position != 0: # если у нас есть позиция, то закрываем ее
                            self.close_trades(cur_price, self.df.index[i])

            if i == len(self.df) - 1 and self.cur_position != 0: # Закрываем сделку на последней свече
                self.close_trades(cur_price, self.df.index[i])
            
            # расчет equity и drowdown
            eq_max = self.df['Equity'].max()
            self.df.loc[self.df.index[i], 'Equity'] = self.equity
            drowdown = (self.equity - eq_max) / eq_max
            self.df.loc[self.df.index[i], 'Drawdown'] = drowdown * 100

    def open_trade(self, trade_type, price, timestamp):
        """
        Открытие сделки с учетом комиссии.

        :param trade_type: Тип сделки ('buy' для покупки, 'sell' для продажи).
        :param price: Цена открытия.
        :param timestamp: Время открытия.
        """
        self.pos_price_open = price

        if trade_type == 'buy':
            self.cur_position = 1
        else:
            self.cur_position = -1

        position_value = self.equity * self.position_size
 
        commission_cost = position_value * self.commission  # Стоимость комиссии

        trade = {
            'Type': trade_type,
            'EntryPrice': price,
            'EntryTime': timestamp,
            'Size': position_value,
            'Commission': commission_cost
        }

        self.equity -= commission_cost  # Уменьшаем баланс на сумму комиссии
        self.positions.append(trade) # Добавляем сделку в список открытых позиций

    def close_trades(self, price, timestamp, stop_l=False):
        """
        Закрытие всех открытых позиций.

        :param price: Цена закрытия.
        :param timestamp: Время закрытия.
        """
        trade = self.positions[-1]
        #print(trade['Size'])
        if trade['Type'] == 'buy':
            pnl = (price - self.pos_price_open) / self.pos_price_open * trade['Size']  # Лонг: разница между ценой выхода и входа
        elif trade['Type'] == 'sell':
            pnl = (self.pos_price_open - price) / self.pos_price_open * trade['Size']  # Шорт: обратная разница

        # Сохранение информации о сделке
        self.trades.append({
            'Type': trade['Type'],
            'EntryPrice': trade['EntryPrice'],
            'ExitPrice': price,
            'EntryTime': trade['EntryTime'],
            'ExitTime': timestamp,
            'PnL': pnl,
            'ReturnPct': pnl / self.equity * 100,
            'Exit_stop': 1 if stop_l else 0
        })

        # Удаляем закрытую позицию
        self.positions.remove(trade)
        self.cur_position = 0 # Обнуляем текущую позицию
        self.equity += pnl # Обновляем эквити
        self.trailing_stop_price = 0.0 # сбрасываем цену для расчета трейлинг-стопа

        # new_data = {'type': trade['Type'], 'entry': self.pos_price_open, 'exit': price, 'pnl': pnl}
        # self.df_result.loc[len(self.df_result)] = new_data
        # print(pnl)

    def _sharp_ratio_calculate(self):
        """Расчет коэффицицента Шарпа"""
        df = self.df.copy()
        
        # Безрисковая ставка
        risk_free_rate = 0.001  # 0.1%

        # Расчет доходности портфеля
        df['Return'] = df['Equity'].pct_change()
        df = df.dropna()

        # Расчет избыточной доходности
        df['ExcessReturn'] = df['Return'] - risk_free_rate

        # Расчет средней избыточной доходности
        mean_excess_return = df['ExcessReturn'].mean()

        # Расчет стандартного отклонения доходности портфеля
        std_return = df['Return'].std()

        # Расчет коэффициента Шарпа
        sharpe_ratio = mean_excess_return / std_return
        return sharpe_ratio

    def calculate_stats(self, only_return=False):
        """
        Расчет статистики стратегии.
        :param only_return - Если True, то будет считать только Return [%] (можно использовать при оптимизации)
        
        :return: Словарь с ключевыми метриками.
        """
        if only_return:
            return {"Return [%]": ((self.equity - self.initial_balance) / self.initial_balance) * 100}

        df_trades = pd.DataFrame(self.trades)
        stats = {
            "Start": self.df.index[0],
            "End": self.df.index[-1],
            "Duration": self.df.index[-1] - self.df.index[0],
            "Equity Start": self.initial_balance,
            "Equity Final": self.equity,
            "Buy and Hold [%]": (self.df['Close'].iloc[-1] - self.df['Close'].iloc[0]) / self.df['Close'].iloc[0] * 100,
            "Return [%]": ((self.equity - self.initial_balance) / self.initial_balance) * 100,
            "Sharp Ratio": self._sharp_ratio_calculate(),
            "Trades": len(df_trades),
            "Win Rate [%]": (df_trades['PnL'] > 0).mean() * 100 if not df_trades.empty else 0,
            "Best Trade [%]": df_trades['ReturnPct'].max() if not df_trades.empty else None,
            "Worst Trade [%]": df_trades['ReturnPct'].min() if not df_trades.empty else None,
            "Profit Factor": df_trades.loc[df_trades['PnL'] > 0, 'PnL'].sum() / abs(df_trades.loc[df_trades['PnL'] < 0, 'PnL'].sum()) if not df_trades.empty else None,
            "Max. Drawdown [%]": self.df['Drawdown'].min(),
        }
        return stats
    
    @staticmethod
    def display_stats(stats):
        """
        Вывод статистики стратегии в удобном формате.
        """
        print("\n" + "=" * 30 + " Strategy Statistics " + "=" * 30)
        for key, value in stats.items():
            print(f"{key:25}: {value}")
        print("=" * 80)

    def plot(self, show_close_trades=False):
        """
        Отображение графиков стратегии с учетом комиссии.

        :param show_close_trades=False - показывать закрытые по стопу сделки
        """
        fig = make_subplots(
            rows=3, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=("Цена и сделки", "Кривая капитала", "Кривая просадок"),
            row_heights=[0.5, 0.3, 0.2]
        )

        # === График 1: Цена и сделки ===
        fig.add_trace(
            go.Candlestick(
                x=self.df.index,
                open=self.df['Open'],
                high=self.df['High'],
                low=self.df['Low'],
                close=self.df['Close'],
                name="Цена"
            ),
            row=1, col=1
        )

        # Добавление SMA, если она есть
        if 'SMA' in self.df.columns:
            fig.add_trace(
                go.Scatter(
                    x=self.df.index,
                    y=self.df['SMA'],
                    mode='lines',
                    name='SMA',
                    line=dict(color='blue', width=2)
                ),
                row=1, col=1
            )

        # Добавление сделок
        for trade in self.trades:
            fig.add_trace(
                go.Scatter(
                    x=[trade['EntryTime']],
                    y=[trade['EntryPrice']],
                    mode='markers',
                    marker=dict(
                        color='green' if trade['Type'] == 'buy' else 'red',
                        size=10,
                        symbol='triangle-up' if trade['Type'] == 'buy' else 'triangle-down'
                    ),
                    name="Сделка"
                ),
                row=1, col=1
            )

            if show_close_trades: # and trade['Exit_stop'] == 1: # для отображения закрытых сделок только по стопу (для реверсных стратегий)
                fig.add_trace(
                    go.Scatter(
                        x=[trade['ExitTime']],
                        y=[trade['ExitPrice']],
                        mode='markers',
                        marker=dict(
                            color='black',
                            size=5,
                            symbol='circle'
                        ),
                        name="Закрытие сделки"
                    ),
                    row=1, col=1
                )

        # === График 2: Кривая капитала ===
        fig.add_trace(
            go.Scatter(
                x=self.df.index,
                y=self.df['Equity'],
                mode='lines',
                name="Капитал",
                line=dict(color='green', width=2)
            ),
            row=2, col=1
        )

        # Установка диапазона y-axis для графика капитала
        equity_min = self.df['Equity'].min()
        equity_max = self.df['Equity'].max()
        fig.update_yaxes(range=[equity_min * 0.95, equity_max * 1.05], row=2, col=1)

        # === График 3: Кривая просадок ===
        fig.add_trace(
            go.Scatter(
                x=self.df.index,
                y=self.df['Drawdown'],
                mode='lines',
                name="Просадка",
                line=dict(color='red', width=2)
            ),
            row=3, col=1
        )

        fig.update_layout(
            title="Результаты стратегии",
            xaxis_title="Дата",
            template="plotly_white",
            showlegend=False,
            height=900
        )
        fig.show()

In [147]:
eth_df_last = yf.download('ETH-USD', start='2025-02-12', end='2025-02-21', interval='15m', multi_level_index=False)
btc_df_last = yf.download('BTC-USD', start='2025-02-12', end='2025-02-21', interval='15m', multi_level_index=False)

eth_df_last['btc_close'] = btc_df_last['Close']
eth_df_last

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Close,High,Low,Open,Volume,btc_close
Datetime,,,,,,
2025-02-12 00:00:00+00:00,2608.576904,2608.697510,2599.905273,2602.628662,0,95942.703125
2025-02-12 00:15:00+00:00,2606.317627,2611.173096,2597.840332,2611.173096,2149672960,95887.992188
2025-02-12 00:30:00+00:00,2601.401855,2611.884033,2595.102539,2606.098145,740081664,95890.476562
2025-02-12 00:45:00+00:00,2612.493652,2612.966309,2601.488525,2602.920166,343044096,96075.570312
2025-02-12 01:00:00+00:00,2604.246338,2613.334717,2604.246338,2610.935791,865075200,95958.265625
...,...,...,...,...,...,...
2025-02-20 22:45:00+00:00,2730.908447,2734.996338,2729.194336,2734.996338,994264064,98201.843750
2025-02-20 23:00:00+00:00,2732.252197,2733.762939,2730.264404,2730.264404,784332800,98317.570312
2025-02-20 23:15:00+00:00,2739.197998,2739.879883,2733.227539,2733.352295,0,98347.953125


In [148]:
eth_df_last_features = prepare_date(eth_df_last, lags=window_lag, windows_SMA=window_SMA, windows_RSI=window_RSI, windows_stats=windows_stats, windows_trend=windows_trend)

Дубликатов не обнаружено.
Обнаружены пропущенные или нулевые значения. Выполняется заполнение предыдущими значениями...


C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\176028078.py:23: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\176028078.py:26: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [149]:
eth_df_last_features

,Close,High,Low,Open,Volume,btc_close,Close_lag_3,High_lag_3,Low_lag_3,Open_lag_3,Volume_lag_3,btc_close_lag_3,Close_lag_9,High_lag_9,Low_lag_9,Open_lag_9,Volume_lag_9,btc_close_lag_9,Close_lag_15,High_lag_15,Low_lag_15,Open_lag_15,Volume_lag_15,btc_close_lag_15,Close_lag_30,High_lag_30,Low_lag_30,Open_lag_30,Volume_lag_30,btc_close_lag_30,Close_SMA_6,Close_SMA_9,Close_SMA_30,Close_SMA_50,Close_RSI_4,Close_RSI_9,Close_RSI_20,Close_mean_3,Close_median_3,Close_min_3,Close_max_3,Close_std_3,Close_range_3,Close_mean_10,Close_median_10,Close_min_10,Close_max_10,Close_std_10,Close_range_10,Close_mean_30,Close_median_30,Close_min_30,Close_max_30,Close_std_30,Close_range_30,Close_mean_50,Close_median_50,Close_min_50,Close_max_50,Close_std_50,Close_range_50,btc_close_mean_3,btc_close_median_3,btc_close_min_3,btc_close_max_3,btc_close_std_3,btc_close_range_3,btc_close_mean_10,btc_close_median_10,btc_close_min_10,btc_close_max_10,btc_close_std_10,btc_close_range_10,btc_close_mean_30,btc_close_median_30,btc_close_min_30,btc_close_max_30,btc_close_std_30,btc_close_range_30,btc_close_mean_50,btc_close_median_50,btc_close_min_50,btc_close_max_50,btc_close_std_50,btc_close_range_50,High-Low,Close_ratio_1,Close_log_diff_1,Close_momentum_3,Close_roc_3,Close_momentum_10,Close_roc_10,Close_momentum_30,Close_roc_30,Close_momentum_50,Close_roc_50,btc_close_ratio_1,btc_close_log_diff_1,btc_close_momentum_3,btc_close_roc_3,btc_close_momentum_10,btc_close_roc_10,btc_close_momentum_30,btc_close_roc_30,btc_close_momentum_50,btc_close_roc_50,Month,Weekday,Hour
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-12 12:30:00+00:00,2634.901367,2634.901367,2628.272217,2631.129395,6.445834e+08,96171.835938,2625.899170,2637.750488,2625.013184,2634.196777,7.667425e+08,96065.937500,2627.677490,2627.677490,2623.248535,2626.074951,6.998835e+07,96110.765625,2628.940430,2631.828369,2626.884766,2627.597168,761319424.0,96325.351562,2602.128418,2603.286865,2588.025146,2589.362549,6.491812e+08,95720.156250,2631.761312,2628.553440,2620.460457,2612.532354,65.602836,60.909918,57.294062,2633.668294,2634.901367,2631.014404,2635.089111,2.300252,4.074707,2628.465845,2628.618774,2616.199219,2635.089111,5.884446,18.889893,2620.460457,2622.223755,2601.137451,2635.089111,10.644039,33.951660,2612.532354,2612.190552,2580.082520,2635.089111,14.387428,55.006592,96207.041667,96192.101562,96171.835938,96257.187500,44.594028,85.351562,96158.834375,96161.433594,95962.484375,96310.882812,98.375765,348.398438,96073.131771,96131.742188,95754.703125,96329.164062,173.075887,574.460938,95939.376094,96006.304688,95292.671875,96329.164062,278.550955,1036.492188,6.629150,0.999929,-0.000071,9.002197,0.342823,8.970703,0.341620,32.772949,1.259467,26.324463,1.009150,0.999113,-0.000887,105.898438,0.110235,28.343750,0.029481,451.679688,0.471875,229.132812,0.238823,2,2,12
2025-02-12 12:45:00+00:00,2633.471924,2636.925537,2633.471924,2636.737061,8.139776e+07,96090.085938,2631.014404,2631.349121,2626.689453,2626.689453,6.948454e+08,96192.101562,2626.928223,2630.908447,2625.579102,2625.579102,1.210757e+09,96149.117188,2621.161865,2629.617188,2620.410889,2629.617188,721901568.0,96199.140625,2606.268555,2606.766846,2600.967285,2602.534180,1.139552e+09,95866.812500,2632.413289,2629.280518,2621.367236,2613.075439,58.903681,58.754311,56.469835,2634.487467,2634.901367,2633.471924,2635.089111,0.884482,1.617188,2629.045288,2630.287231,2616.199219,2635.089111,6.080224,18.889893,2621.367236,2624.592407,2601.137451,2635.089111,10.551666,33.951660,2613.075439,2613.226318,2580.082520,2635.089111,14.658007,55.006592,96173.036458,96171.835938,96090.085938,96257.187500,83.557250,167.101562,96156.766406,96161.433594,95962.484375,96310.882812,99.706847,348.398438,96080.574219,96131.742188,95754.703125,96329.164062,168.641705,574.460938,95943.417969,96021.140625,95292.671875,96329.164062,279.255476,1036.492188,3.453613,0.999457,-0.000543,2.457520,0.0934

## Подготовим предсказания ML

In [150]:
pred_xgboost = best_model_xgboost.predict(eth_df_last_features)
prob_xgboost = best_model_xgboost.predict_proba(eth_df_last_features)

pred_LGBMC = best_model_LGBMC.predict(eth_df_last_features)
prob_LGBMC = best_model_LGBMC.predict_proba(eth_df_last_features)

In [151]:
print(pred_xgboost.shape, pred_LGBMC.shape)
print(prob_xgboost.shape, prob_LGBMC.shape)

(814,) (814,)
(814, 3) (814, 3)


## Подготовка предсказаний для NN

In [152]:
eth_df_last_scale, _ = normalize_dataframe(eth_df_last_features, scaler=scaler_NN)
eth_df_last_scale

,Close,High,Low,Open,Volume,btc_close,Close_lag_3,High_lag_3,Low_lag_3,Open_lag_3,Volume_lag_3,btc_close_lag_3,Close_lag_9,High_lag_9,Low_lag_9,Open_lag_9,Volume_lag_9,btc_close_lag_9,Close_lag_15,High_lag_15,Low_lag_15,Open_lag_15,Volume_lag_15,btc_close_lag_15,Close_lag_30,High_lag_30,Low_lag_30,Open_lag_30,Volume_lag_30,btc_close_lag_30,Close_SMA_6,Close_SMA_9,Close_SMA_30,Close_SMA_50,Close_RSI_4,Close_RSI_9,Close_RSI_20,Close_mean_3,Close_median_3,Close_min_3,Close_max_3,Close_std_3,Close_range_3,Close_mean_10,Close_median_10,Close_min_10,Close_max_10,Close_std_10,Close_range_10,Close_mean_30,Close_median_30,Close_min_30,Close_max_30,Close_std_30,Close_range_30,Close_mean_50,Close_median_50,Close_min_50,Close_max_50,Close_std_50,Close_range_50,btc_close_mean_3,btc_close_median_3,btc_close_min_3,btc_close_max_3,btc_close_std_3,btc_close_range_3,btc_close_mean_10,btc_close_median_10,btc_close_min_10,btc_close_max_10,btc_close_std_10,btc_close_range_10,btc_close_mean_30,btc_close_median_30,btc_close_min_30,btc_close_max_30,btc_close_std_30,btc_close_range_30,btc_close_mean_50,btc_close_median_50,btc_close_min_50,btc_close_max_50,btc_close_std_50,btc_close_range_50,High-Low,Close_ratio_1,Close_log_diff_1,Close_momentum_3,Close_roc_3,Close_momentum_10,Close_roc_10,Close_momentum_30,Close_roc_30,Close_momentum_50,Close_roc_50,btc_close_ratio_1,btc_close_log_diff_1,btc_close_momentum_3,btc_close_roc_3,btc_close_momentum_10,btc_close_roc_10,btc_close_momentum_30,btc_close_roc_30,btc_close_momentum_50,btc_close_roc_50,Month,Weekday,Hour
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-12 12:30:00+00:00,-1.721059,-1.746361,-1.717013,-1.734360,0.357013,-0.829002,-1.753860,-1.740427,-1.730965,-1.727685,0.514941,-0.855595,-1.754539,-1.779961,-1.743179,-1.760722,-0.383159,-0.841894,-1.756820,-1.772782,-1.737699,-1.762134,0.509283,-0.784330,-1.860507,-1.882704,-1.880425,-1.903465,0.367653,-0.935313,-1.735735,-1.748708,-1.793102,-1.836451,0.733211,0.748992,0.731850,-1.726941,-1.722612,-1.706369,-1.750808,-0.705279,-0.731142,-1.749829,-1.749626,-1.706418,-1.806558,-0.658059,-0.677116,-1.793102,-1.788696,-1.673612,-1.901167,-0.708025,-0.885166,-1.836451,-1.841263,-1.687962,-1.974641,-0.714357,-0.824253,-0.820655,-0.824471,-0.790441,-0.845273,-0.737189,-0.735255,-0.834697,-0.833499,-0.774349,-0.897164,-0.784116,-0.754115,-0.861115,-0.846168,-0.706121,-0.993308,-0.844951,-0.993993,-0.900836,-0.882510,-0.738674,-1.066134,-0.797759,-0.951392,-0.500802,-0.005101,-0.002901,0.406668,0.461829,0.247773,0.275345,0.484488,0.552293,0.352378,0.389272,-0.318002,-0.316761,0.208222,0.214382,0.025001,0.021425,0.278540,0.282965,0.101998,0.096183,1.546811,-0.500256,0.071241
2025-02-12 12:45:00+00:00,-1.725724,-1.739735,-1.700087,-1.716049,-0.369623,-0.850150,-1.737148,-1.761401,-1.725503,-1.752224,0.422179,-0.822974,-1.756992,-1.769355,-1.735571,-1.762346,1.088533,-0.831990,-1.782329,-1.780054,-1.758871,-1.755507,0.458432,-0.816878,-1.846872,-1.871209,-1.837925,-1.860070,1.000798,-0.897600,-1.733604,-1.746328,-1.790114,-1.834650,0.423565,0.605386,0.653498,-1.724266,-1.722612,-1.698366,-1.750808,-0.890197,-0.899716,-1.747932,-1.744166,-1.706418,-1.806558,-0.641650,-0.677116,-1.790114,-1.780893,-1.673612,-1.901167,-0.712744,-0.885166,-1.834650,-1.837827,-1.687962,-1.974641,-0.703524,-0.824253,-0.829461,-0.829717,-0.811664,-0.845273,-0.500324,-0.476310,-0.835234,-0.833499,-0.774349,-0.897164,-0.778735,-0.754115,-0.859169,-0.846168,-0.706121,-0.993308,-0.856242,-0.993993,-0.899771,-0.878624,-0.738674,-1.066134,-0.796300,-0.951392,-0.763398,-0.113776,-0.110996,0.127027,0.139317,0.173783,0.189320,0.412834,0.466637,0.360890,0.399659,-0.304975,-0.303721,-0.206356,-0.217271,-0.030143,-0.035897,0.131476,0.129173,0.088001,0.081691,1.546811,-0.500256,0.071241
2025-02-12 13:00:00+00:00,-1.724035,-1.749345,-1.722135,-1.738772,0.160571,-0.896297,-1.723835,-1.732822,-1.711718,-1.738395,-0.432490,-0

In [153]:
eth_df_last_scale = create_windows_for_predict(eth_df_last_scale, window_size=window_size)
eth_df_last_scale.shape

(784, 30, 109)

In [154]:
eth_df_last_scale = torch.from_numpy(eth_df_last_scale).float() # Преобразуем в тензор

In [155]:
eth_df_last_scale = eth_df_last_scale.to(device)

In [156]:
pred_TS_attention, prob_TS_attention = get_prediction(TS_attention_model, eth_df_last_scale)
pred_res_net_2, prob_res_net_2 = get_prediction(res_net_model_tr_2, eth_df_last_scale)
pred_res_net_3, prob_res_net_3 = get_prediction(res_net_model_tr_3, eth_df_last_scale)

In [157]:
print(pred_TS_attention.shape, prob_TS_attention.shape)
print(pred_res_net_2.shape, prob_res_net_2.shape)
print(pred_res_net_3.shape, prob_res_net_3.shape)

(784,) (784, 3)
(784,) (784, 3)
(784,) (784, 3)


## Объединим предсказания в датасет

In [158]:
# Обрежем предсказания ML моделей на длину окна для нейронных сетей
pred_xgboost = pred_xgboost[30:]
prob_xgboost = prob_xgboost[30:]
pred_LGBMC = pred_LGBMC[30:]
prob_LGBMC = prob_LGBMC[30:]

# Создаём DataFrame
trade_ensemble_df = pd.DataFrame(index=eth_df_last_features.index[30:])

# Добавляем предсказания XGBoost
trade_ensemble_df['xgboost_pred'] = pred_xgboost
trade_ensemble_df['xgboost_prob_class_0'] = prob_xgboost[:, 0]
trade_ensemble_df['xgboost_prob_class_1'] = prob_xgboost[:, 1]
trade_ensemble_df['xgboost_prob_class_2'] = prob_xgboost[:, 2]

# Добавляем предсказания LightGBM
trade_ensemble_df['lightgbm_pred'] = pred_LGBMC
trade_ensemble_df['lightgbm_prob_class_0'] = prob_LGBMC[:, 0]
trade_ensemble_df['lightgbm_prob_class_1'] = prob_LGBMC[:, 1]
trade_ensemble_df['lightgbm_prob_class_2'] = prob_LGBMC[:, 2]

# Добавляем предсказания TS Attention
trade_ensemble_df['ts_attention_pred'] = pred_TS_attention
trade_ensemble_df['ts_attention_prob_class_0'] = prob_TS_attention[:, 0]
trade_ensemble_df['ts_attention_prob_class_1'] = prob_TS_attention[:, 1]
trade_ensemble_df['ts_attention_prob_class_2'] = prob_TS_attention[:, 2]

# Добавляем предсказания ResNet 2
trade_ensemble_df['resnet_2_pred'] = pred_res_net_2
trade_ensemble_df['resnet_2_prob_class_0'] = prob_res_net_2[:, 0]
trade_ensemble_df['resnet_2_prob_class_1'] = prob_res_net_2[:, 1]
trade_ensemble_df['resnet_2_prob_class_2'] = prob_res_net_2[:, 2]

# Добавляем предсказания ResNet 3
trade_ensemble_df['resnet_3_pred'] = pred_res_net_3
trade_ensemble_df['resnet_3_prob_class_0'] = prob_res_net_3[:, 0]
trade_ensemble_df['resnet_3_prob_class_1'] = prob_res_net_3[:, 1]
trade_ensemble_df['resnet_3_prob_class_2'] = prob_res_net_3[:, 2]


In [159]:
trade_ensemble_df

,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2
Datetime,,,,,,,,,,,,,,,,,,,,
2025-02-12 20:00:00+00:00,0,0.491129,0.043607,0.465263,2,0.361874,0.243584,0.394542,2,0.379478,0.184249,0.436273,2,0.158922,0.116583,0.724495,1,0.152908,0.577677,0.269415
2025-02-12 20:15:00+00:00,0,0.474200,0.096247,0.429552,2,0.369527,0.227587,0.402886,2,0.363095,0.197448,0.439457,2,0.225726,0.162935,0.611339,1,0.321449,0.520612,0.157939
2025-02-12 20:30:00+00:00,2,0.149522,0.185064,0.665414,2,0.330758,0.241233,0.428009,2,0.308455,0.129286,0.562259,2,0.199806,0.155413,0.644780,1,0.179652,0.596658,0.223690
2025-02-12 20:45:00+00:00,2,0.361947,0.157548,0.480505,2,0.333141,0.227803,0.439056,2,0.288537,0.071608,0.639855,2,0.083275,0.060742,0.855983,2,0.042579,0.338261,0.619160
2025-02-12 21:00:00+00:00,0,0.409103,0.204951,0.385945,2,0.333141,0.227803,0.439056,2,0.281633,0.075796,0.642572,2,0.100359,0.087784,0.811857,1,0.068429,0.485121,0.446450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-20 22:45:00+00:00,0,0.679042,0.087435,0.233524,0,0.406587,0.232837,0.360577,2,0.325047,0.242952,0.432001,0,0.906565,0.078392,0.015043,0,0.987270,0.011322,0.001408
2025-02-20 23:00:00+00:00,0,0.739301,0.131813,0.128885,0,0.433166,0.248058,0.318776,2,0.334840,0.253587,0.411573,0,0.896662,0.085473,0.017865,0,0.985487,0.012786,0.001727
2025-02-20 23:15:00+00:00,0,0.557721,0.318755,0.123524,0,0.359517,0.320547,0.319935,2,0.337373,0.265479,0.397149,0,0.911786,0.074369,0.013845,0,0.988354,0.010332,0.001315


In [160]:
def making_signal_by_col(df, column_name):
    "Функция, преобразовывает классы в сигналы (-1, 0, 1)"
    df = df.copy()

    # Создаем колонку Signal на основе предсказаний модели
    df['Signal'] = np.where(df[column_name] == 0, -1, np.where(df[column_name] == 1, 0, 1))
    return df


In [161]:
# Объеденим датафреймы, чтобы можно было провести бэктест

# Объединяем по индексу
trade_df = trade_ensemble_df.join(eth_df_last, how='inner')

In [162]:
trade_df

,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2,Close,High,Low,Open,Volume,btc_close
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-12 20:00:00+00:00,0,0.491129,0.043607,0.465263,2,0.361874,0.243584,0.394542,2,0.379478,0.184249,0.436273,2,0.158922,0.116583,0.724495,1,0.152908,0.577677,0.269415,2687.499512,2687.499512,2671.068848,2677.491943,0,97422.445312
2025-02-12 20:15:00+00:00,0,0.474200,0.096247,0.429552,2,0.369527,0.227587,0.402886,2,0.363095,0.197448,0.439457,2,0.225726,0.162935,0.611339,1,0.321449,0.520612,0.157939,2688.991211,2692.365479,2687.932617,2691.302002,379486208,97539.890625
2025-02-12 20:30:00+00:00,2,0.149522,0.185064,0.665414,2,0.330758,0.241233,0.428009,2,0.308455,0.129286,0.562259,2,0.199806,0.155413,0.644780,1,0.179652,0.596658,0.223690,2672.340820,2685.933594,2672.340820,2685.933594,132669440,97067.585938
2025-02-12 20:45:00+00:00,2,0.361947,0.157548,0.480505,2,0.333141,0.227803,0.439056,2,0.288537,0.071608,0.639855,2,0.083275,0.060742,0.855983,2,0.042579,0.338261,0.619160,2679.118896,2679.118896,2673.629883,2673.802979,1099282432,97034.890625
2025-02-12 21:00:00+00:00,0,0.409103,0.204951,0.385945,2,0.333141,0.227803,0.439056,2,0.281633,0.075796,0.642572,2,0.100359,0.087784,0.811857,1,0.068429,0.485121,0.446450,2685.450928,2685.450928,2675.414062,2676.786865,0,97367.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-20 22:45:00+00:00,0,0.679042,0.087435,0.233524,0,0.406587,0.232837,0.360577,2,0.325047,0.242952,0.432001,0,0.906565,0.078392,0.015043,0,0.987270,0.011322,0.001408,2730.908447,2734.996338,2729.194336,2734.996338,994264064,98201.843750
2025-02-20 23:00:00+00:00,0,0.739301,0.131813,0.128885,0,0.433166,0.248058,0.318776,2,0.334840,0.253587,0.411573,0,0.896662,0.085473,0.017865,0,0.985487,0.012786,0.001727,2732.252197,2733.762939,2730.264404,2730.264404,784332800,98317.570312
2025-02-20 23:15:00+00:00,0,0.557721,0.318755,0.123524,0,0.359517,0.320547,0.319935,2,0.337373,0.265479,0.397149,0,0.911786,0.074369,0.013845,0,0.988354,0.010332,0.001315,2739.197998,2739.879883,2733.227539,2733.352295,0,98347.953125


# Voting

## Hard Voting

In [66]:
# Попробуем объединить предсказания классов моделей. Выберем класс, который предсказало большинство моделей.

In [67]:
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

def evaluate_metrics(y_true, y_pred, y_prob=None, label="Results"):
    """
    Вычисляет accuracy, F1-score, precision, recall, ROC-AUC для каждого класса и их macro-усреднение.
    """
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)

    # Метрики для каждого класса
    f1 = f1_score(y_true, y_pred, average=None)
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)

    # Macro-усреднение
    f1_macro = f1_score(y_true, y_pred, average='macro')
    precision_macro = precision_score(y_true, y_pred, average='macro')
    recall_macro = recall_score(y_true, y_pred, average='macro')

    # ROC-AUC (если переданы вероятности)
    if y_prob is not None:
        roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr')
    else:
        roc_auc = None

    # Вывод результатов
    print(f"Metrics for {label}:")
    print(f"Accuracy = {accuracy:.4f}")
    for i, (f1_val, prec_val, rec_val) in enumerate(zip(f1, precision, recall)):
        print(f"Class {i}: F1 = {f1_val:.4f}, Precision = {prec_val:.4f}, Recall = {rec_val:.4f}")
    print(f"Macro: F1 = {f1_macro:.4f}, Precision = {precision_macro:.4f}, Recall = {recall_macro:.4f}")
    if roc_auc is not None:
        print(f"ROC-AUC (OvR) = {roc_auc:.4f}")
    print()

In [68]:
def hard_voting(df, columns):
    """
    Принимает на вход датасет с предсказаниями моделей.
    Возвращает итоговое предсказание путём голосования.
    """
    # Извлекаем предсказания всех моделей
    model_predictions = df[columns].values

    # Применяем Hard Voting
    from scipy.stats import mode
    return mode(model_predictions, axis=1)[0].flatten()

In [69]:
# Применяем Hard Voting к train, val и test
train_final_pred = hard_voting(train_ensemble_df, columns_pred)
val_final_pred = hard_voting(val_ensemble_df, columns_pred)
test_final_pred = hard_voting(test_ensemble_df, columns_pred)

In [70]:
# Оценка на train
evaluate_metrics(
    train_ensemble_df['y_true'], 
    train_final_pred, 
    y_prob=None,
    label="Train"
)

# Оценка на val
evaluate_metrics(
    val_ensemble_df['y_true'], 
    val_final_pred, 
    y_prob=None,
    label="Validation"
)

# Оценка на test
evaluate_metrics(
    test_ensemble_df['y_true'], 
    test_final_pred, 
    y_prob=None,
    label="Test"
)

Metrics for Train:
Accuracy = 0.6170
Class 0: F1 = 0.6495, Precision = 0.4908, Recall = 0.9598
Class 1: F1 = 0.5134, Precision = 0.9757, Recall = 0.3484
Class 2: F1 = 0.6304, Precision = 0.8716, Recall = 0.4938
Macro: F1 = 0.5978, Precision = 0.7794, Recall = 0.6006

Metrics for Validation:
Accuracy = 0.4486
Class 0: F1 = 0.4789, Precision = 0.4000, Recall = 0.5965
Class 1: F1 = 0.2086, Precision = 0.4857, Recall = 0.1328
Class 2: F1 = 0.5529, Precision = 0.4909, Recall = 0.6328
Macro: F1 = 0.4135, Precision = 0.4589, Recall = 0.4540

Metrics for Test:
Accuracy = 0.3865
Class 0: F1 = 0.4874, Precision = 0.3750, Recall = 0.6960
Class 1: F1 = 0.1324, Precision = 0.2500, Recall = 0.0900
Class 2: F1 = 0.3806, Precision = 0.4608, Recall = 0.3241
Macro: F1 = 0.3334, Precision = 0.3619, Recall = 0.3700



## Оценка на бэктесте

In [71]:
trade_final_pred = trade_df.copy()

# Применяем Hard Voting
trade_final_pred['total_pred'] = hard_voting(trade_df, columns_pred)

In [72]:
trade_final_pred = making_signal_by_col(trade_final_pred, 'total_pred')

In [73]:
trade_final_pred

,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2,Close,High,Low,Open,Volume,btc_close,total_pred,Signal
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-12 20:00:00+00:00,0,0.491129,0.043607,0.465263,2,0.361874,0.243584,0.394542,2,0.379478,0.184249,0.436273,2,0.158922,0.116583,0.724495,1,0.152908,0.577677,0.269415,2687.499512,2687.499512,2671.068848,2677.491943,0,97422.445312,2,1
2025-02-12 20:15:00+00:00,0,0.474200,0.096247,0.429552,2,0.369527,0.227587,0.402886,2,0.363095,0.197448,0.439457,2,0.225726,0.162935,0.611339,1,0.321449,0.520612,0.157939,2688.991211,2692.365479,2687.932617,2691.302002,379486208,97539.890625,2,1
2025-02-12 20:30:00+00:00,2,0.149522,0.185064,0.665414,2,0.330758,0.241233,0.428009,2,0.308455,0.129286,0.562259,2,0.199806,0.155413,0.644780,1,0.179652,0.596658,0.223690,2672.340820,2685.933594,2672.340820,2685.933594,132669440,97067.585938,2,1
2025-02-12 20:45:00+00:00,2,0.361947,0.157548,0.480505,2,0.333141,0.227803,0.439056,2,0.288537,0.071608,0.639855,2,0.083275,0.060742,0.855983,2,0.042579,0.338261,0.619160,2679.118896,2679.118896,2673.629883,2673.802979,1099282432,97034.890625,2,1
2025-02-12 21:00:00+00:00,0,0.409103,0.204951,0.385945,2,0.333141,0.227803,0.439056,2,0.281633,0.075796,0.642572,2,0.100359,0.087784,0.811857,1,0.068429,0.485121,0.446450,2685.450928,2685.450928,2675.414062,2676.786865,0,97367.250000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-20 22:45:00+00:00,0,0.679042,0.087435,0.233524,0,0.406587,0.232837,0.360577,2,0.325047,0.242952,0.432001,0,0.906565,0.078392,0.015043,0,0.987270,0.011322,0.001408,2730.908447,2734.996338,2729.194336,2734.996338,994264064,98201.843750,0,-1
2025-02-20 23:00:00+00:00,0,0.739301,0.131813,0.128885,0,0.433166,0.248058,0.318776,2,0.334840,0.253587,0.411573,0,0.896662,0.085473,0.017865,0,0.985487,0.012786,0.001727,2732.252197,2733.762939,2730.264404,2730.264404,784332800,98317.570312,0,-1
2025-02-20 23:15:00+00:00,0,0.557721,0.318755,0.123524,0,0.359517,0.320547,0.319935,2,0.337373,0.265479,0.397149,0,0.911786,0.074369,0.013845,0,0.988354,0.010332,0.001315,2739.197998,2739.879883,2733.227539,2733.352295,0,98347.953125,0,-1


In [74]:
bt = StrategyTester(trade_final_pred, initial_balance=100000, position_size=0.1)

# Запускаем бэктест
bt.run()
stats = bt.calculate_stats()
StrategyTester.display_stats(stats)

bt.plot()

C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\4293038873.py:135: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.df.loc[self.df.index[i], 'Drawdown'] = drowdown * 100
C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\4293038873.py:133: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '99991.99405919739' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.df.loc[self.df.index[i], 'Equity'] = self.equity



============================== Strategy Statistics ==============================
Start                    : 2025-02-12 20:00:00+00:00
End                      : 2025-02-20 23:45:00+00:00
Duration                 : 8 days 03:45:00
Equity Start             : 100000
Equity Final             : 100679.18129114046
Buy and Hold [%]         : 1.9894989079758192
Return [%]               : 0.6791812911404559
Sharp Ratio              : -4.786533974764564
Trades                   : 71
Win Rate [%]             : 59.154929577464785
Best Trade [%]           : 0.2489757679110233
Worst Trade [%]          : -0.24592178073229257
Profit Factor            : 2.496344150498883
Max. Drawdown [%]        : -0.2749528451690572


## Soft Voting

In [76]:
def soft_voting(df, columns_class_0, columns_class_1, columns_class_2, threshold=0):
    """
    Принимает на вход датафрейм с вероятностями классов для каждой модели.
    Возвращает итоговое предсказание путём усреднения вероятностей (Soft Voting).
    
    Параметры:
    - df: Датафрейм с предсказаниями моделей.
    - columns_class_0: Список колонок с вероятностями для класса 0.
    - columns_class_1: Список колонок с вероятностями для класса 1.
    - columns_class_2: Список колонок с вероятностями для класса 2.
    - threshold: Порог для выбора класса 0 или 2. Если средняя вероятность > threshold, выбираем класс 0 или 2.
                Если threshold = 0, выбираем класс с наибольшей средней вероятностью.
    
    Возвращает:
    - Итоговые предсказания (классы с наибольшей средней вероятностью или с учётом порога).
    """
    # Усредняем вероятности для каждого класса
    avg_prob_class_0 = df[columns_class_0].mean(axis=1)
    avg_prob_class_1 = df[columns_class_1].mean(axis=1)
    avg_prob_class_2 = df[columns_class_2].mean(axis=1)

    # Собираем усреднённые вероятности в один массив
    avg_probabilities = np.vstack([avg_prob_class_0, avg_prob_class_1, avg_prob_class_2]).T

    if threshold == 0:
        # Если threshold = 0, выбираем класс с наибольшей средней вероятностью
        final_predictions = np.argmax(avg_probabilities, axis=1)
    else:
        # Иначе применяем порог
        final_predictions = np.ones(len(df), dtype=int)  # По умолчанию класс 1

        # Если средняя вероятность для класса 0 > threshold, выбираем класс 0
        final_predictions[avg_prob_class_0 > threshold] = 0

        # Если средняя вероятность для класса 2 > threshold, выбираем класс 2
        final_predictions[avg_prob_class_2 > threshold] = 2

    return final_predictions

In [77]:
# Применяем Soft Voting к train, val и test
train_final_pred = soft_voting(train_ensemble_df, columns_class_0, columns_class_1, columns_class_2)
val_final_pred = soft_voting(val_ensemble_df, columns_class_0, columns_class_1, columns_class_2)
test_final_pred = soft_voting(test_ensemble_df, columns_class_0, columns_class_1, columns_class_2)

In [78]:
# Оценка на train
evaluate_metrics(
    train_ensemble_df['y_true'], 
    train_final_pred, 
    y_prob=None,
    label="Train"
)

# Оценка на val
evaluate_metrics(
    val_ensemble_df['y_true'], 
    val_final_pred, 
    y_prob=None,
    label="Validation"
)

# Оценка на test
evaluate_metrics(
    test_ensemble_df['y_true'], 
    test_final_pred, 
    y_prob=None,
    label="Test"
)

Metrics for Train:
Accuracy = 0.3881
Class 0: F1 = 0.5294, Precision = 0.3688, Recall = 0.9383
Class 1: F1 = 0.0406, Precision = 0.8571, Recall = 0.0208
Class 2: F1 = 0.2183, Precision = 0.5539, Recall = 0.1359
Macro: F1 = 0.2628, Precision = 0.5933, Recall = 0.3650

Metrics for Validation:
Accuracy = 0.4243
Class 0: F1 = 0.4721, Precision = 0.3770, Recall = 0.6316
Class 1: F1 = 0.0709, Precision = 0.3846, Recall = 0.0391
Class 2: F1 = 0.5442, Precision = 0.4819, Recall = 0.6250
Macro: F1 = 0.3624, Precision = 0.4145, Recall = 0.4319

Metrics for Test:
Accuracy = 0.3622
Class 0: F1 = 0.4527, Precision = 0.3527, Recall = 0.6320
Class 1: F1 = 0.0794, Precision = 0.1923, Recall = 0.0500
Class 2: F1 = 0.3774, Precision = 0.4167, Recall = 0.3448
Macro: F1 = 0.3031, Precision = 0.3206, Recall = 0.3423



In [91]:
# Применяем Soft Voting к train, val и test с трешхолдом
threshold = 0.6

train_final_pred = soft_voting(train_ensemble_df, columns_class_0, columns_class_1, columns_class_2, threshold=threshold)
val_final_pred = soft_voting(val_ensemble_df, columns_class_0, columns_class_1, columns_class_2, threshold=threshold)
test_final_pred = soft_voting(test_ensemble_df, columns_class_0, columns_class_1, columns_class_2, threshold=threshold)

In [92]:
# Оценка на train
evaluate_metrics(
    train_ensemble_df['y_true'], 
    train_final_pred, 
    y_prob=None,
    label="Train"
)

# Оценка на val
evaluate_metrics(
    val_ensemble_df['y_true'], 
    val_final_pred, 
    y_prob=None,
    label="Validation"
)

# Оценка на test
evaluate_metrics(
    test_ensemble_df['y_true'], 
    test_final_pred, 
    y_prob=None,
    label="Test"
)

Metrics for Train:
Accuracy = 0.6316
Class 0: F1 = 0.9075, Precision = 0.9216, Recall = 0.8938
Class 1: F1 = 0.6114, Precision = 0.4524, Recall = 0.9428
Class 2: F1 = 0.1785, Precision = 0.8882, Recall = 0.0992
Macro: F1 = 0.5658, Precision = 0.7540, Recall = 0.6453

Metrics for Validation:
Accuracy = 0.4216
Class 0: F1 = 0.1460, Precision = 0.4348, Recall = 0.0877
Class 1: F1 = 0.5026, Precision = 0.3780, Recall = 0.7500
Class 2: F1 = 0.4525, Precision = 0.5376, Recall = 0.3906
Macro: F1 = 0.3670, Precision = 0.4501, Recall = 0.4094

Metrics for Test:
Accuracy = 0.3216
Class 0: F1 = 0.1893, Precision = 0.3636, Recall = 0.1280
Class 1: F1 = 0.4372, Precision = 0.2919, Recall = 0.8700
Class 2: F1 = 0.1850, Precision = 0.5714, Recall = 0.1103
Macro: F1 = 0.2705, Precision = 0.4090, Recall = 0.3694



## Оценка на бэктесте

In [95]:
trade_final_pred = trade_df.copy()

# Применяем Soft Voting
trade_final_pred['total_pred'] = soft_voting(trade_df, columns_class_0, columns_class_1, columns_class_2)

In [96]:
trade_final_pred = making_signal_by_col(trade_final_pred, 'total_pred')

In [97]:
trade_final_pred

,xgboost_pred,xgboost_prob_class_0,xgboost_prob_class_1,xgboost_prob_class_2,lightgbm_pred,lightgbm_prob_class_0,lightgbm_prob_class_1,lightgbm_prob_class_2,ts_attention_pred,ts_attention_prob_class_0,ts_attention_prob_class_1,ts_attention_prob_class_2,resnet_2_pred,resnet_2_prob_class_0,resnet_2_prob_class_1,resnet_2_prob_class_2,resnet_3_pred,resnet_3_prob_class_0,resnet_3_prob_class_1,resnet_3_prob_class_2,Close,High,Low,Open,Volume,btc_close,total_pred,Signal
Datetime,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-02-12 20:00:00+00:00,0,0.491129,0.043607,0.465263,2,0.361874,0.243584,0.394542,2,0.379478,0.184249,0.436273,2,0.158922,0.116583,0.724495,1,0.152908,0.577677,0.269415,2687.499512,2687.499512,2671.068848,2677.491943,0,97422.445312,2,1
2025-02-12 20:15:00+00:00,0,0.474200,0.096247,0.429552,2,0.369527,0.227587,0.402886,2,0.363095,0.197448,0.439457,2,0.225726,0.162935,0.611339,1,0.321449,0.520612,0.157939,2688.991211,2692.365479,2687.932617,2691.302002,379486208,97539.890625,2,1
2025-02-12 20:30:00+00:00,2,0.149522,0.185064,0.665414,2,0.330758,0.241233,0.428009,2,0.308455,0.129286,0.562259,2,0.199806,0.155413,0.644780,1,0.179652,0.596658,0.223690,2672.340820,2685.933594,2672.340820,2685.933594,132669440,97067.585938,2,1
2025-02-12 20:45:00+00:00,2,0.361947,0.157548,0.480505,2,0.333141,0.227803,0.439056,2,0.288537,0.071608,0.639855,2,0.083275,0.060742,0.855983,2,0.042579,0.338261,0.619160,2679.118896,2679.118896,2673.629883,2673.802979,1099282432,97034.890625,2,1
2025-02-12 21:00:00+00:00,0,0.409103,0.204951,0.385945,2,0.333141,0.227803,0.439056,2,0.281633,0.075796,0.642572,2,0.100359,0.087784,0.811857,1,0.068429,0.485121,0.446450,2685.450928,2685.450928,2675.414062,2676.786865,0,97367.250000,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-20 22:45:00+00:00,0,0.679042,0.087435,0.233524,0,0.406587,0.232837,0.360577,2,0.325047,0.242952,0.432001,0,0.906565,0.078392,0.015043,0,0.987270,0.011322,0.001408,2730.908447,2734.996338,2729.194336,2734.996338,994264064,98201.843750,0,-1
2025-02-20 23:00:00+00:00,0,0.739301,0.131813,0.128885,0,0.433166,0.248058,0.318776,2,0.334840,0.253587,0.411573,0,0.896662,0.085473,0.017865,0,0.985487,0.012786,0.001727,2732.252197,2733.762939,2730.264404,2730.264404,784332800,98317.570312,0,-1
2025-02-20 23:15:00+00:00,0,0.557721,0.318755,0.123524,0,0.359517,0.320547,0.319935,2,0.337373,0.265479,0.397149,0,0.911786,0.074369,0.013845,0,0.988354,0.010332,0.001315,2739.197998,2739.879883,2733.227539,2733.352295,0,98347.953125,0,-1


In [98]:
bt = StrategyTester(trade_final_pred, initial_balance=100000, position_size=0.1)

# Запускаем бэктест
bt.run()
stats = bt.calculate_stats()
StrategyTester.display_stats(stats)

bt.plot()

C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\4293038873.py:135: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\Vit\AppData\Local\Temp\ipykernel_3448\4293038873.py:133: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100016.28948350021' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.




============================== Strategy Statistics ==============================
Start                    : 2025-02-12 20:00:00+00:00
End                      : 2025-02-20 23:45:00+00:00
Duration                 : 8 days 03:45:00
Equity Start             : 100000
Equity Final             : 100183.36211430792
Buy and Hold [%]         : 1.9894989079758192
Return [%]               : 0.1833621143079217
Sharp Ratio              : -9.542816899322915
Trades                   : 37
Win Rate [%]             : 62.16216216216216
Best Trade [%]           : 0.15280644314238154
Worst Trade [%]          : -0.07879965325961175
Profit Factor            : 2.238614744274332
Max. Drawdown [%]        : -0.28625127268178246


# Stacking

## Мета модель на нейронных сетях

In [103]:
from sklearn.metrics import classification_report, roc_auc_score

# Функция для вывода метрик
def evaluate_model(y_true, y_pred, y_prob, name):
    print(f"\n=== Метрики для {name} выборки ===")
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=['Class 0', 'Class 1', 'Class 2']))
    
    # ROC AUC (для многоклассовой классификации)
    roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro')
    print(f"ROC AUC (OvR, macro): {roc_auc:.4f}")

In [104]:
def get_predictions(model, dataloader, device):
    '''Функция для получения предсказаний и вероятностей'''
    
    model.eval()  # Переводим модель в режим оценки
    y_true = []
    y_pred = []
    y_prob = []
    
    with torch.no_grad():  # Отключаем вычисление градиентов
        for batch in dataloader:
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)
            
            # Forward pass
            outputs = model(features)
            
            # Получаем предсказанные классы
            _, predicted = torch.max(outputs, 1)
            
            # Получаем вероятности классов
            prob = torch.softmax(outputs, dim=1)
            
            # Сохраняем результаты
            y_true.extend(torch.argmax(labels, dim=1).cpu().numpy())  # Преобразуем one-hot в индексы классов
            y_pred.extend(predicted.cpu().numpy())
            y_prob.extend(prob.cpu().numpy())
    
    return np.array(y_true), np.array(y_pred), np.array(y_prob)

In [116]:
def train_model(
    model,  # Модель
    train_loader,  # DataLoader для обучающей выборки
    val_loader,  # DataLoader для валидационной выборки
    criterion,  # Функция потерь
    optimizer,  # Оптимизатор
    scheduler,  # Шедулер
    num_epochs,  # Количество эпох
    device,  # Устройство (CPU или GPU)
    name_model,  # Название модели для сохранения весов
):
    # Инициализация переменных для сохранения лучших моделей
    best_train_accuracy = 0.0
    best_val_accuracy = 0.0
    best_val_loss = float("inf")

    # Цикл обучения
    for epoch in range(num_epochs):
        model.train()  # Переводим модель в режим обучения

        train_loss = 0.0
        train_correct = 0
        train_total = 0

        # Обучение на обучающей выборке
        for batch_idx, batch in enumerate(train_loader):
            # Перемещаем батч на устройство (GPU или CPU)
            features, labels = batch
            features = features.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, torch.argmax(labels, dim=1))  # labels в one-hot, преобразуем в классы

            # Backward pass и оптимизация
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Считаем loss и accuracy
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

        # Валидация
        model.eval()  # Переводим модель в режим оценки
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():  # Отключаем вычисление градиентов
            for batch in val_loader:
                # Перемещаем батч на устройство (GPU или CPU)
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(features)
                loss = criterion(outputs, torch.argmax(labels, dim=1))

                # Считаем loss и accuracy
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

        # Вычисляем средние loss и accuracy
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        train_accuracy = 100 * train_correct / train_total
        val_accuracy = 100 * val_correct / val_total

        # Вывод логов
        print(
            f"Epoch [{epoch + 1}/{num_epochs}], "
            f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, "
            f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%"
        )

        # Обновление шедулера
        scheduler.step(val_loss)

        # Сохранение лучшей модели по train accuracy
        if train_accuracy > best_train_accuracy:
            best_train_accuracy = train_accuracy
            torch.save(model.state_dict(), f'{name_model}_train_best_acc.pth')
            print(f"Best train model saved with Train Accuracy: {train_accuracy:.2f}%")

        # Сохранение лучшей модели по val accuracy
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), f'{name_model}_val_best_acc.pth')
            print(f"Best val model saved with Val Accuracy: {val_accuracy:.2f}%")

        
        # Сохранение лучшей модели по val loss
        # if val_loss < best_val_loss:
        #     best_val_loss = val_loss
        #     print(f"Best val loss: {val_loss:.4f}")

    print("Training complete!")

In [109]:
# Функция для преобразования датафрейма в тензоры
def prepare_data(df):
    """
    Преобразует датафрейм с предсказаниями моделей в тензоры PyTorch.
    
    Параметры:
    - df: Датафрейм с предсказаниями моделей.
    
    Возвращает:
    - features: Тензор с признаками (вероятности классов от всех моделей).
    - labels: Тензор с целевыми метками в one-hot encoding.
    """
    # Выделяем фичи (вероятности классов)
    feature_columns = [col for col in df.columns if 'prob_class_' in col]
    features = df[feature_columns].values

    # Преобразуем в тензор PyTorch
    features = torch.tensor(features, dtype=torch.float32)

    # Извлекаем целевые метки и преобразуем в one-hot encoding
    labels = df['y_true'].values
    labels = torch.tensor(labels, dtype=torch.long)
    labels = F.one_hot(labels, num_classes=3).float()  # One-hot encoding

    return features, labels

In [110]:
# Подготовка данных
train_features, train_labels = prepare_data(train_ensemble_df)
val_features, val_labels = prepare_data(val_ensemble_df)
test_features, test_labels = prepare_data(test_ensemble_df)

In [111]:
print(train_features.shape, train_labels.shape)
print(val_features.shape, val_labels.shape)
print(test_features.shape, test_labels.shape)

torch.Size([3909, 15]) torch.Size([3909, 3])
torch.Size([370, 15]) torch.Size([370, 3])
torch.Size([370, 15]) torch.Size([370, 3])


In [134]:
# Создаём Dataset
train_dataset = TensorDataset(train_features, train_labels)
val_dataset = TensorDataset(val_features, val_labels)
test_dataset = TensorDataset(test_features, test_labels)

# Создаём DataLoader
batch_size = 32  # Размер мини-батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [135]:
class MetaClassifier(nn.Module):
    """
    Простая полносвязная нейронная сеть для стекинга.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MetaClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Параметры модели
input_size = len([col for col in train_ensemble_df.columns if 'prob_class_' in col])  # Количество признаков
hidden_size = 64  # Размер скрытого слоя
output_size = 3  # Количество классов

# Создаём модель
meta_model_dense = MetaClassifier(input_size, hidden_size, output_size)
meta_model_dense.to(device)

MetaClassifier(
  (fc1): Linear(in_features=15, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

In [136]:
# Гиперпараметры
learning_rate = 0.001
num_epochs = 50
patience = 5  # Количество эпох для уменьшения loss

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(meta_model_dense.parameters(), lr=learning_rate)

# Шедулер для уменьшения learning rate
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=True)

train_model(meta_model_dense, train_loader, val_loader, 
            criterion=criterion, optimizer=optimizer, 
            scheduler=scheduler, num_epochs=num_epochs, 
            device=device, name_model='meta_model_dense')

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.



Epoch [1/50], Train Loss: 0.8874, Train Accuracy: 72.99%, Val Loss: 1.1015, Val Accuracy: 41.89%
Best train model saved with Train Accuracy: 72.99%
Best val model saved with Val Accuracy: 41.89%
Epoch [2/50], Train Loss: 0.3818, Train Accuracy: 99.77%, Val Loss: 1.2103, Val Accuracy: 44.32%
Best train model saved with Train Accuracy: 99.77%
Best val model saved with Val Accuracy: 44.32%
Epoch [3/50], Train Loss: 0.1116, Train Accuracy: 100.00%, Val Loss: 1.4147, Val Accuracy: 42.97%
Best train model saved with Train Accuracy: 100.00%
Epoch [4/50], Train Loss: 0.0448, Train Accuracy: 100.00%, Val Loss: 1.5814, Val Accuracy: 43.78%
Epoch [5/50], Train Loss: 0.0249, Train Accuracy: 99.97%, Val Loss: 1.7158, Val Accuracy: 43.24%
Epoch [6/50], Train Loss: 0.0163, Train Accuracy: 100.00%, Val Loss: 1.8284, Val Accuracy: 43.51%
Epoch [7/50], Train Loss: 0.0117, Train Accuracy: 100.00%, Val Loss: 1.9256, Val Accuracy: 43.51%
Epoch [8/50], Train Loss: 0.0101, Train Accuracy: 99.97%, Val Loss: 1

In [137]:
# Получаем предсказания для обучающей, валидационной и тестовой выборок
y_true_train, y_pred_train, y_prob_train = get_predictions(meta_model_dense, train_loader, device)
y_true_val, y_pred_val, y_prob_val = get_predictions(meta_model_dense, val_loader, device)
y_true_test, y_pred_test, y_prob_test = get_predictions(meta_model_dense, test_loader, device)

# Оцениваем модель
evaluate_model(y_true_train, y_pred_train, y_prob_train, "обучающей")
evaluate_model(y_true_val, y_pred_val, y_prob_val, "валидационной")
evaluate_model(y_true_test, y_pred_test, y_prob_test, "тестовой")


=== Метрики для обучающей выборки ===

Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00      1394
     Class 1       1.00      1.00      1.00      1154
     Class 2       1.00      1.00      1.00      1361

    accuracy                           1.00      3909
   macro avg       1.00      1.00      1.00      3909
weighted avg       1.00      1.00      1.00      3909

ROC AUC (OvR, macro): 1.0000

=== Метрики для валидационной выборки ===

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.40      0.42      0.41       114
     Class 1       0.52      0.27      0.35       128
     Class 2       0.44      0.64      0.52       128

    accuracy                           0.44       370
   macro avg       0.46      0.44      0.43       370
weighted avg       0.46      0.44      0.43       370

ROC AUC (OvR, macro): 0.6007

=== Метрики для тестовой выборки ===

Classificat

In [138]:
class MetaClassifier(nn.Module):
    """
    Улучшенная модель с Dropout и Batch Normalization.
    """
    def __init__(self, input_size, hidden_size, output_size):
        super(MetaClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)  # Batch Normalization
        self.dropout = nn.Dropout(0.7)  # Dropout
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)  # Применяем Dropout
        x = self.fc2(x)
        return x
    
# Создаём модель
meta_model_dense_2 = MetaClassifier(input_size, hidden_size, output_size)
meta_model_dense_2.to(device)

MetaClassifier(
  (fc1): Linear(in_features=15, out_features=64, bias=True)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.7, inplace=False)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)

In [139]:
hidden_size = 8  # Размер скрытого слоя

# Функция потерь и оптимизатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(meta_model_dense_2.parameters(), lr=learning_rate, weight_decay=1e-5)

# Шедулер для уменьшения learning rate
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=patience, verbose=True)

train_model(meta_model_dense_2, train_loader, val_loader, 
            criterion=criterion, optimizer=optimizer, 
            scheduler=scheduler, num_epochs=num_epochs, 
            device=device, name_model='meta_model_dense_2')

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning:

The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.



Epoch [1/50], Train Loss: 0.4113, Train Accuracy: 88.82%, Val Loss: 1.3216, Val Accuracy: 42.70%
Best train model saved with Train Accuracy: 88.82%
Best val model saved with Val Accuracy: 42.70%
Epoch [2/50], Train Loss: 0.1226, Train Accuracy: 98.70%, Val Loss: 1.6128, Val Accuracy: 43.51%
Best train model saved with Train Accuracy: 98.70%
Best val model saved with Val Accuracy: 43.51%
Epoch [3/50], Train Loss: 0.0712, Train Accuracy: 99.21%, Val Loss: 1.8531, Val Accuracy: 43.51%
Best train model saved with Train Accuracy: 99.21%
Epoch [4/50], Train Loss: 0.0528, Train Accuracy: 99.31%, Val Loss: 2.0522, Val Accuracy: 43.24%
Best train model saved with Train Accuracy: 99.31%
Epoch [5/50], Train Loss: 0.0424, Train Accuracy: 99.36%, Val Loss: 2.2339, Val Accuracy: 43.78%
Best train model saved with Train Accuracy: 99.36%
Best val model saved with Val Accuracy: 43.78%
Epoch [6/50], Train Loss: 0.0339, Train Accuracy: 99.59%, Val Loss: 2.4194, Val Accuracy: 44.05%
Best train model saved

In [140]:
# Получаем предсказания для обучающей, валидационной и тестовой выборок
y_true_train, y_pred_train, y_prob_train = get_predictions(meta_model_dense_2, train_loader, device)
y_true_val, y_pred_val, y_prob_val = get_predictions(meta_model_dense_2, val_loader, device)
y_true_test, y_pred_test, y_prob_test = get_predictions(meta_model_dense_2, test_loader, device)

# Оцениваем модель
evaluate_model(y_true_train, y_pred_train, y_prob_train, "обучающей")
evaluate_model(y_true_val, y_pred_val, y_prob_val, "валидационной")
evaluate_model(y_true_test, y_pred_test, y_prob_test, "тестовой")


=== Метрики для обучающей выборки ===

Classification Report:
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00      1394
     Class 1       1.00      1.00      1.00      1154
     Class 2       1.00      1.00      1.00      1361

    accuracy                           1.00      3909
   macro avg       1.00      1.00      1.00      3909
weighted avg       1.00      1.00      1.00      3909

ROC AUC (OvR, macro): 1.0000

=== Метрики для валидационной выборки ===

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.39      0.46      0.42       114
     Class 1       0.50      0.28      0.36       128
     Class 2       0.44      0.56      0.49       128

    accuracy                           0.43       370
   macro avg       0.44      0.43      0.42       370
weighted avg       0.44      0.43      0.42       370

ROC AUC (OvR, macro): 0.6009

=== Метрики для тестовой выборки ===

Classificat

## Логистическая регрессия

In [179]:
# Функция для подготовки данных
def prepare_data(df):
    """
    Преобразует датафрейм с предсказаниями моделей в фичи и метки.
    
    Параметры:
    - df: Датафрейм с предсказаниями моделей.
    
    Возвращает:
    - features: Массив с признаками (вероятности классов от всех моделей).
    - labels: Массив с целевыми метками.
    """
    # Выделяем фичи (вероятности классов)
    feature_columns = [col for col in df.columns if 'prob_class_' in col]
    features = df[feature_columns].values

    # Извлекаем целевые метки
    labels = df['y_true'].values

    return features, labels

# Подготовка данных
X_train, y_train = prepare_data(train_ensemble_df)
X_val, y_val = prepare_data(val_ensemble_df)
X_test, y_test = prepare_data(test_ensemble_df)

In [182]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Создаём модель логистической регрессии
meta_model = LogisticRegression(penalty='l2', C=0.1, multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Обучаем модель на обучающей выборке
meta_model.fit(X_train, y_train)

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.



LogisticRegression(C=0.1, max_iter=1000, multi_class='multinomial')

In [183]:
def evaluate_model_LR(model, X, y, label="Results"):
    """
    Оценивает качество модели на данных.
    
    Параметры:
    - model: Обученная модель.
    - X: Признаки.
    - y: Истинные метки.
    - label: Название выборки (для вывода).
    """
    # Предсказания модели
    y_pred = model.predict(X)

    # Метрики
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='macro')
    precision = precision_score(y, y_pred, average='macro')
    recall = recall_score(y, y_pred, average='macro')

    # Вывод результатов
    print(f"Metrics for {label}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-score (macro): {f1:.4f}")
    print(f"Precision (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print()

# Оценка на обучающей выборке
evaluate_model_LR(meta_model, X_train, y_train, label="Train")

# Оценка на валидационной выборке
evaluate_model_LR(meta_model, X_val, y_val, label="Validation")

# Оценка на тестовой выборке
evaluate_model_LR(meta_model, X_test, y_test, label="Test")

Metrics for Train:
Accuracy: 1.0000
F1-score (macro): 1.0000
Precision (macro): 1.0000
Recall (macro): 1.0000

Metrics for Validation:
Accuracy: 0.4432
F1-score (macro): 0.4264
Precision (macro): 0.4588
Recall (macro): 0.4424

Metrics for Test:
Accuracy: 0.3568
F1-score (macro): 0.3414
Precision (macro): 0.3447
Recall (macro): 0.3424



In [185]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2']}
grid_search = GridSearchCV(LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000), param_grid, cv=3)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'C': 0.1, 'penalty': 'l2'}


c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning:

'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.

c:\Users\Vit\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logi

## Решающие деревья

In [186]:
from sklearn.tree import DecisionTreeClassifier

# Создаём модель решающего дерева
tree_model = DecisionTreeClassifier(max_depth=5, random_state=42)

# Обучаем модель на обучающей выборке
tree_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [188]:
# Оценка на обучающей выборке
evaluate_model_LR(tree_model, X_train, y_train, label="Train")

# Оценка на валидационной выборке
evaluate_model_LR(tree_model, X_val, y_val, label="Validation")

# Оценка на тестовой выборке
evaluate_model_LR(tree_model, X_test, y_test, label="Test")

Metrics for Train:
Accuracy: 1.0000
F1-score (macro): 1.0000
Precision (macro): 1.0000
Recall (macro): 1.0000

Metrics for Validation:
Accuracy: 0.4297
F1-score (macro): 0.4194
Precision (macro): 0.4262
Recall (macro): 0.4256

Metrics for Test:
Accuracy: 0.3405
F1-score (macro): 0.3323
Precision (macro): 0.3446
Recall (macro): 0.3393



Выводы: лучше всего себя показал ансамбль Hard Voiting. Метамодели Stacking очень сильно переобучались.